In [2]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 13.7 MB/s 
     |████████████████████████████████| 210 kB 75.5 MB/s 
     |████████████████████████████████| 81 kB 11.7 MB/s 
     |████████████████████████████████| 78 kB 9.7 MB/s 
     |████████████████████████████████| 50 kB 8.0 MB/s 
     |████████████████████████████████| 147 kB 62.4 MB/s 
     |████████████████████████████████| 112 kB 72.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=10c128c8c259a64360f2b09ffdf4ea63f38d2356bb8ca073bb7028557ed55f3b
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.1.0
    Uninstalling importlib-metadata-5.1.0:
      Successfully uninstalled importlib-metadata-5.1.0


In [3]:
import optuna

from lightgbm import LGBMRegressor
from optuna.samplers import TPESampler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold

from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import random
import os
from tqdm import tqdm
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

import warnings
warnings.filterwarnings('ignore')

def MAE(true, pred):
    mae = np.mean(np.abs(true-pred))
    return mae

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/real_estate/final_dataset.csv")

In [7]:
#동만 수치화
qual_col = ["법정동"]

for i in qual_col:
  le = LabelEncoder()
  le = le.fit(df[i])
  df[i] = le.transform(df[i])

In [8]:
Jongno_df = df[df["구"] == "종로구"] #구 선택
Jongno_x = Jongno_df.drop(columns = ["구" , "면적당가격(만원)"]) #구 단위로 선택했기 때문에 구 drop
Jongno_y = Jongno_df["면적당가격(만원)"]

Jung_df = df[df["구"] == "중구"]
Jung_x = Jung_df.drop(columns = ["구" , "면적당가격(만원)"])
Jung_y = Jung_df["면적당가격(만원)"]

Yongsan_df = df[df["구"] == "용산구"]
Yongsan_x = Yongsan_df.drop(columns = ["구" , "면적당가격(만원)"])
Yongsan_y = Yongsan_df["면적당가격(만원)"]

Seongdong_df = df[df["구"] == "성동구"]
Seongdong_x = Seongdong_df.drop(columns = ["구" , "면적당가격(만원)"])
Seongdong_y = Seongdong_df["면적당가격(만원)"]

Gwangjin_df = df[df["구"] == "광진구"]
Gwangjin_x = Gwangjin_df.drop(columns = ["구" , "면적당가격(만원)"])
Gwangjin_y = Gwangjin_df["면적당가격(만원)"]

Dongdaemun_df = df[df["구"] == "동대문구"]
Dongdaemun_x = Dongdaemun_df.drop(columns = ["구" , "면적당가격(만원)"])
Dongdaemun_y = Dongdaemun_df["면적당가격(만원)"]

Jungnang_df = df[df["구"] == "중랑구"]
Jungnang_x = Jungnang_df.drop(columns = ["구" , "면적당가격(만원)"])
Jungnang_y = Jungnang_df["면적당가격(만원)"]

Seongbuk_df = df[df["구"] == "성북구"]
Seongbuk_x = Seongbuk_df.drop(columns = ["구" , "면적당가격(만원)"])
Seongbuk_y = Seongbuk_df["면적당가격(만원)"]

Gangbuk_df = df[df["구"] == "강북구"]
Gangbuk_x = Gangbuk_df.drop(columns = ["구" , "면적당가격(만원)"])
Gangbuk_y = Gangbuk_df["면적당가격(만원)"]

Dobong_df = df[df["구"] == "도봉구"]
Dobong_x = Dobong_df.drop(columns = ["구" , "면적당가격(만원)"])
Dobong_y = Dobong_df["면적당가격(만원)"]

Nowon_df = df[df["구"] == "노원구"]
Nowon_x = Nowon_df.drop(columns = ["구" , "면적당가격(만원)"])
Nowon_y = Nowon_df["면적당가격(만원)"]

Eunpyeong_df = df[df["구"] == "은평구"]
Eunpyeong_x = Eunpyeong_df.drop(columns = ["구" , "면적당가격(만원)"])
Eunpyeong_y = Eunpyeong_df["면적당가격(만원)"]

Seodaemun_df = df[df["구"] == "서대문구"]
Seodaemun_x = Seodaemun_df.drop(columns = ["구" , "면적당가격(만원)"])
Seodaemun_y = Seodaemun_df["면적당가격(만원)"]

Mapo_df = df[df["구"] == "마포구"]
Mapo_x = Mapo_df.drop(columns = ["구" , "면적당가격(만원)"])
Mapo_y = Mapo_df["면적당가격(만원)"]

Yangcheon_df = df[df["구"] == "양천구"]
Yangcheon_x = Yangcheon_df.drop(columns = ["구" , "면적당가격(만원)"])
Yangcheon_y = Yangcheon_df["면적당가격(만원)"]

Gangseo_df = df[df["구"] == "강서구"]
Gangseo_x = Gangseo_df.drop(columns = ["구" , "면적당가격(만원)"])
Gangseo_y = Gangseo_df["면적당가격(만원)"]

Guro_df = df[df["구"] == "구로구"]
Guro_x = Guro_df.drop(columns = ["구" , "면적당가격(만원)"])
Guro_y = Guro_df["면적당가격(만원)"]

Geumcheon_df = df[df["구"] == "금천구"]
Geumcheon_x = Geumcheon_df.drop(columns = ["구" , "면적당가격(만원)"])
Geumcheon_y = Geumcheon_df["면적당가격(만원)"]

Yeongdeungpo_df = df[df["구"] == "영등포구"]
Yeongdeungpo_x = Yeongdeungpo_df.drop(columns = ["구" , "면적당가격(만원)"])
Yeongdeungpo_y = Yeongdeungpo_df["면적당가격(만원)"]

Dongjak_df = df[df["구"] == "동작구"]
Dongjak_x = Dongjak_df.drop(columns = ["구" , "면적당가격(만원)"])
Dongjak_y = Dongjak_df["면적당가격(만원)"]

Gwanak_df = df[df["구"] == "관악구"]
Gwanak_x = Gwanak_df.drop(columns = ["구" , "면적당가격(만원)"])
Gwanak_y = Gwanak_df["면적당가격(만원)"]

Seocho_df = df[df["구"] == "서초구"]
Seocho_x = Seocho_df.drop(columns = ["구" , "면적당가격(만원)"])
Seocho_y = Seocho_df["면적당가격(만원)"]

Gangnam_df = df[df["구"] == "강남구"]
Gangnam_x = Gangnam_df.drop(columns = ["구" , "면적당가격(만원)"])
Gangnam_y = Gangnam_df["면적당가격(만원)"]

Songpa_df = df[df["구"] == "송파구"]
Songpa_x = Songpa_df.drop(columns = ["구" , "면적당가격(만원)"])
Songpa_y = Songpa_df["면적당가격(만원)"]

Gangdong_df = df[df["구"] == "강동구"]
Gangdong_x = Gangdong_df.drop(columns = ["구" , "면적당가격(만원)"])
Gangdong_y = Gangdong_df["면적당가격(만원)"]

### GridSearchCV로 넓은 구간의 하이퍼파라미터 탐색

In [60]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Jongno_x, Jongno_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 200}
LGBMRegressor(n_estimators=200)


### Kfold + Optuna로 최적 하이퍼파라미터 도출

In [61]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

n_iter=0
for train_index, test_index in tqdm(kfold.split(Jongno_x, Jongno_y)):
    n_iter += 1
    print(n_iter)
    train_x= Jongno_x.iloc[train_index]
    valid_x= Jongno_x.iloc[test_index]
    train_y= Jongno_y.iloc[train_index]
    valid_y= Jongno_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

#for model Jongno
#Best trial : score 209.3640002033729, 
#params {'learning_rate': 0.020329055792006407, 'n_estimators': 245, 'colsample_bytree': 0.6403861812204279, 'subsample': 0.5121579832157269, 'reg_alpha': 3.227361479535839, 'reg_lambda': 0.8855533970352447}

0it [00:00, ?it/s][I 2022-12-17 08:24:30,309] A new study created in memory with name: no-name-9a1e2aa7-ba79-454e-87da-f3d7ace0590f
[I 2022-12-17 08:24:30,346] Trial 0 finished with value: 374.5778530575949 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 291, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 374.5778530575949.
[I 2022-12-17 08:24:30,379] Trial 1 finished with value: 435.61092120548096 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 274, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 435.61092120548096.
[I 2022-12-17 08:24:30,402] Trial 2 finished with value: 374.12155250192916 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 142, 'colsample_bytree': 0.590912483603

1


[I 2022-12-17 08:24:30,517] Trial 6 finished with value: 398.88687377903454 and parameters: {'learning_rate': 0.0311567631481637, 'n_estimators': 119, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}. Best is trial 1 with value: 435.61092120548096.
[I 2022-12-17 08:24:30,550] Trial 7 finished with value: 457.01036926326157 and parameters: {'learning_rate': 0.004404463590406622, 'n_estimators': 282, 'colsample_bytree': 0.6293899908000085, 'subsample': 0.831261142176991, 'reg_alpha': 1.5585553804470549, 'reg_lambda': 2.600340105889054}. Best is trial 7 with value: 457.01036926326157.
[I 2022-12-17 08:24:30,575] Trial 8 finished with value: 377.1670136376122 and parameters: {'learning_rate': 0.05512431765498469, 'n_estimators': 137, 'colsample_bytree': 0.9847923138822793, 'subsample': 0.8875664116805573, 'reg_alpha': 4.697494707820946, 'reg_lambda': 4.474136752138244}. Best is trial 7 with value: 457

Best trial : score 457.01036926326157, 
params {'learning_rate': 0.004404463590406622, 'n_estimators': 282, 'colsample_bytree': 0.6293899908000085, 'subsample': 0.831261142176991, 'reg_alpha': 1.5585553804470549, 'reg_lambda': 2.600340105889054}
2


[I 2022-12-17 08:24:30,842] Trial 5 finished with value: 276.8903580765119 and parameters: {'learning_rate': 0.012839830347891868, 'n_estimators': 243, 'colsample_bytree': 0.8803925243084487, 'subsample': 0.7806385987847482, 'reg_alpha': 3.854835899772805, 'reg_lambda': 2.4689779818219537}. Best is trial 2 with value: 389.380578925579.
[I 2022-12-17 08:24:30,870] Trial 6 finished with value: 257.7382184985521 and parameters: {'learning_rate': 0.052750550108817414, 'n_estimators': 185, 'colsample_bytree': 0.5127095633720475, 'subsample': 0.5539457134966522, 'reg_alpha': 0.15714592843367126, 'reg_lambda': 3.182052056318902}. Best is trial 2 with value: 389.380578925579.
[I 2022-12-17 08:24:30,900] Trial 7 finished with value: 261.1421474926499 and parameters: {'learning_rate': 0.03212124212655634, 'n_estimators': 202, 'colsample_bytree': 0.9537832369630466, 'subsample': 0.6246461145744375, 'reg_alpha': 2.0519146151781484, 'reg_lambda': 3.7777556927152434}. Best is trial 2 with value: 389

...

Best trial : score 301.39683957028603, 
params {'learning_rate': 0.005036739013921628, 'n_estimators': 218, 'colsample_bytree': 0.8387821809211412, 'subsample': 0.508293914463928, 'reg_alpha': 2.560465291496405, 'reg_lambda': 1.1324788759896898}
4


[I 2022-12-17 08:24:31,503] Trial 6 finished with value: 203.46178592311657 and parameters: {'learning_rate': 0.03321457011776751, 'n_estimators': 250, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}. Best is trial 3 with value: 208.40309103489366.
[I 2022-12-17 08:24:31,529] Trial 7 finished with value: 202.7840697895359 and parameters: {'learning_rate': 0.010273802014981157, 'n_estimators': 173, 'colsample_bytree': 0.6326011838408627, 'subsample': 0.6219948216895418, 'reg_alpha': 4.865052773762228, 'reg_lambda': 1.965488623333802}. Best is trial 3 with value: 208.40309103489366.
[I 2022-12-17 08:24:31,560] Trial 8 finished with value: 201.12560316373077 and parameters: {'learning_rate': 0.08931260896253422, 'n_estimators': 226, 'colsample_bytree': 0.8974056517708242, 'subsample': 0.751318546552596, 'reg_alpha': 2.8845194231317954, 'reg_lambda': 2.4625884690943196}. Best is trial 3 with value: 20

Best trial : score 209.3640002033729, 
params {'learning_rate': 0.020329055792006407, 'n_estimators': 245, 'colsample_bytree': 0.6403861812204279, 'subsample': 0.5121579832157269, 'reg_alpha': 3.227361479535839, 'reg_lambda': 0.8855533970352447}
5


[I 2022-12-17 08:24:31,826] Trial 5 finished with value: 307.51684346195964 and parameters: {'learning_rate': 0.08112675439237285, 'n_estimators': 262, 'colsample_bytree': 0.9335361592900518, 'subsample': 0.9566202762782356, 'reg_alpha': 2.556711994304689, 'reg_lambda': 2.507581473435998}. Best is trial 2 with value: 314.31260048720713.
[I 2022-12-17 08:24:31,857] Trial 6 finished with value: 306.2953325371255 and parameters: {'learning_rate': 0.08003122271771075, 'n_estimators': 230, 'colsample_bytree': 0.8509834386288517, 'subsample': 0.8978963347180505, 'reg_alpha': 4.450026709087831, 'reg_lambda': 1.689975784257679}. Best is trial 2 with value: 314.31260048720713.
[I 2022-12-17 08:24:31,880] Trial 7 finished with value: 284.0000761706114 and parameters: {'learning_rate': 0.03818271231135446, 'n_estimators': 118, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}. Best is trial 2 with value: 314.31

Best trial : score 314.31260048720713, 
params {'learning_rate': 0.08526253048016884, 'n_estimators': 163, 'colsample_bytree': 0.5847463733430462, 'subsample': 0.7784006312291751, 'reg_alpha': 4.680773870803905, 'reg_lambda': 3.480148983374865}


### 다른 구에 대해서 반복

In [11]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Jung_x, Jung_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [12]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Jung_x, Jung_y)):
    n_iter += 1
    print(n_iter)
    train_x= Jung_x.iloc[train_index]
    valid_x= Jung_x.iloc[test_index]
    train_y= Jung_y.iloc[train_index]
    valid_y= Jung_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

#for model Jung
#Best trial : score 307.49241754207577, 
#params {'learning_rate': 0.0015466895952366377, 'n_estimators': 163, 'colsample_bytree': 0.8534286719238086, 'subsample': 0.8645035840204937, 'reg_alpha': 3.8563517334297286, 'reg_lambda': 0.3702232586704518}

0it [00:00, ?it/s][I 2022-12-17 08:03:32,119] A new study created in memory with name: no-name-42c5339d-d495-4de1-bada-0db356c62b8d
[I 2022-12-17 08:03:32,172] Trial 0 finished with value: 272.1031820109531 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 272.1031820109531.


1


[I 2022-12-17 08:03:32,298] Trial 1 finished with value: 388.00937210624943 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 388.00937210624943.
[I 2022-12-17 08:03:32,388] Trial 2 finished with value: 328.56286258609543 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.5909124836035503, 'subsample': 0.5917022549267169, 'reg_alpha': 1.5212112147976886, 'reg_lambda': 2.6237821581611893}. Best is trial 1 with value: 388.00937210624943.
[I 2022-12-17 08:03:32,508] Trial 3 finished with value: 323.0204538581575 and parameters: {'learning_rate': 0.04376255684556946, 'n_estimators': 58, 'colsample_bytree': 0.8059264473611898, 'subsample': 0.569746930326021, 'reg_alpha': 1.4607232426760908, 'reg_lambda': 1.8318092164684585}. Best is trial 1 with value:

Best trial : score 436.2027248000397, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:03:33,878] Trial 1 finished with value: 240.4308961091645 and parameters: {'learning_rate': 0.014951498272501501, 'n_estimators': 161, 'colsample_bytree': 0.5372753218398854, 'subsample': 0.9934434683002586, 'reg_alpha': 3.861223846483287, 'reg_lambda': 0.993578407670862}. Best is trial 1 with value: 240.4308961091645.
[I 2022-12-17 08:03:34,002] Trial 2 finished with value: 307.49241754207577 and parameters: {'learning_rate': 0.0015466895952366377, 'n_estimators': 163, 'colsample_bytree': 0.8534286719238086, 'subsample': 0.8645035840204937, 'reg_alpha': 3.8563517334297286, 'reg_lambda': 0.3702232586704518}. Best is trial 2 with value: 307.49241754207577.
[I 2022-12-17 08:03:34,076] Trial 3 finished with value: 253.72532727350654 and parameters: {'learning_rate': 0.03648810712588299, 'n_estimators': 23, 'colsample_bytree': 0.9315517129377968, 'subsample': 0.811649063413779, 'reg_alpha': 1.654490124263246, 'reg_lambda': 0.3177917514301182}. Best is trial 2 with value: 3

...

Best trial : score 358.42280627516027, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:03:35,580] Trial 6 finished with value: 316.1074380213408 and parameters: {'learning_rate': 0.03321457011776751, 'n_estimators': 150, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}. Best is trial 4 with value: 519.2996914058907.
[I 2022-12-17 08:03:35,602] Trial 7 finished with value: 432.0225485083313 and parameters: {'learning_rate': 0.010273802014981157, 'n_estimators': 73, 'colsample_bytree': 0.6326011838408627, 'subsample': 0.6219948216895418, 'reg_alpha': 4.865052773762228, 'reg_lambda': 1.965488623333802}. Best is trial 4 with value: 519.2996914058907.
[I 2022-12-17 08:03:35,630] Trial 8 finished with value: 295.4260462738765 and parameters: {'learning_rate': 0.08931260896253422, 'n_estimators': 126, 'colsample_bytree': 0.8974056517708242, 'subsample': 0.751318546552596, 'reg_alpha': 2.8845194231317954, 'reg_lambda': 2.4625884690943196}. Best is trial 4 with value: 519.299

Best trial : score 519.2996914058907, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:03:35,879] Trial 6 finished with value: 247.50667400188746 and parameters: {'learning_rate': 0.08003122271771075, 'n_estimators': 130, 'colsample_bytree': 0.8509834386288517, 'subsample': 0.8978963347180505, 'reg_alpha': 4.450026709087831, 'reg_lambda': 1.689975784257679}. Best is trial 3 with value: 358.29882613163835.
[I 2022-12-17 08:03:35,893] Trial 7 finished with value: 380.9299556286257 and parameters: {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}. Best is trial 7 with value: 380.9299556286257.
[I 2022-12-17 08:03:35,920] Trial 8 finished with value: 296.20978254662856 and parameters: {'learning_rate': 0.029367583960700158, 'n_estimators': 118, 'colsample_bytree': 0.5152501249695247, 'subsample': 0.5186740943746072, 'reg_alpha': 4.113002803298292, 'reg_lambda': 1.8009532070563146}. Best is trial 7 with value: 380.9

Best trial : score 380.9299556286257, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [13]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Yongsan_x, Yongsan_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [14]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Yongsan_x, Yongsan_y)):
    n_iter += 1
    print(n_iter)
    train_x= Yongsan_x.iloc[train_index]
    valid_x= Yongsan_x.iloc[test_index]
    train_y= Yongsan_y.iloc[train_index]
    valid_y= Yongsan_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

#for model Yongsan
#Best trial : score 483.63563360175965, 
#params {'learning_rate': 0.03321457011776751, 'n_estimators': 150, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}

0it [00:00, ?it/s][I 2022-12-17 08:03:40,330] A new study created in memory with name: no-name-1619b259-5151-406a-9372-a27b329f56c6
[I 2022-12-17 08:03:40,360] Trial 0 finished with value: 396.62866210044155 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 396.62866210044155.
[I 2022-12-17 08:03:40,388] Trial 1 finished with value: 486.133106602961 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 486.133106602961.
[I 2022-12-17 08:03:40,405] Trial 2 finished with value: 428.9186402869318 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.5909124836035503

1


[I 2022-12-17 08:03:40,515] Trial 7 finished with value: 469.57896324809496 and parameters: {'learning_rate': 0.004404463590406622, 'n_estimators': 182, 'colsample_bytree': 0.6293899908000085, 'subsample': 0.831261142176991, 'reg_alpha': 1.5585553804470549, 'reg_lambda': 2.600340105889054}. Best is trial 1 with value: 486.133106602961.
[I 2022-12-17 08:03:40,535] Trial 8 finished with value: 465.15341340980996 and parameters: {'learning_rate': 0.05512431765498469, 'n_estimators': 37, 'colsample_bytree': 0.9847923138822793, 'subsample': 0.8875664116805573, 'reg_alpha': 4.697494707820946, 'reg_lambda': 4.474136752138244}. Best is trial 1 with value: 486.133106602961.
[I 2022-12-17 08:03:40,564] Trial 9 finished with value: 400.8118950354007 and parameters: {'learning_rate': 0.06019209790229743, 'n_estimators': 185, 'colsample_bytree': 0.5442462510259598, 'subsample': 0.5979914312095727, 'reg_alpha': 0.22613644455269033, 'reg_lambda': 1.6266516538163218}. Best is trial 1 with value: 486.1

Best trial : score 486.133106602961, 
params {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}
2


[I 2022-12-17 08:03:40,791] Trial 7 finished with value: 489.0137962453431 and parameters: {'learning_rate': 0.03212124212655634, 'n_estimators': 102, 'colsample_bytree': 0.9537832369630466, 'subsample': 0.6246461145744375, 'reg_alpha': 2.0519146151781484, 'reg_lambda': 3.7777556927152434}. Best is trial 6 with value: 530.6553350695505.
[I 2022-12-17 08:03:40,806] Trial 8 finished with value: 482.5775838622434 and parameters: {'learning_rate': 0.023651018383670626, 'n_estimators': 15, 'colsample_bytree': 0.6448757264568841, 'subsample': 0.5806106436270022, 'reg_alpha': 4.648488261712865, 'reg_lambda': 4.040601897822085}. Best is trial 6 with value: 530.6553350695505.
[I 2022-12-17 08:03:40,837] Trial 9 finished with value: 524.545336839223 and parameters: {'learning_rate': 0.06370697189453192, 'n_estimators': 175, 'colsample_bytree': 0.9018360384495572, 'subsample': 0.5932850294430179, 'reg_alpha': 4.462794992449889, 'reg_lambda': 2.6967112095782535}. Best is trial 6 with value: 530.65

...

Best trial : score 896.7214310118221, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:03:41,313] Trial 8 finished with value: 455.8796558839451 and parameters: {'learning_rate': 0.08931260896253422, 'n_estimators': 126, 'colsample_bytree': 0.8974056517708242, 'subsample': 0.751318546552596, 'reg_alpha': 2.8845194231317954, 'reg_lambda': 2.4625884690943196}. Best is trial 6 with value: 483.63563360175965.
[I 2022-12-17 08:03:41,338] Trial 9 finished with value: 468.3986207246471 and parameters: {'learning_rate': 0.020329055792006407, 'n_estimators': 145, 'colsample_bytree': 0.6403861812204279, 'subsample': 0.5121579832157269, 'reg_alpha': 3.227361479535839, 'reg_lambda': 0.8855533970352447}. Best is trial 6 with value: 483.63563360175965.
4it [00:01,  3.96it/s][I 2022-12-17 08:03:41,345] A new study created in memory with name: no-name-9cba1ad1-e856-4acf-a214-29bee26fa627
[I 2022-12-17 08:03:41,375] Trial 0 finished with value: 436.7716649788692 and parameters: {'learning_rate': 0.09410539985093852, 'n_estimators': 191, 'colsample_bytree': 0.957432195110

Best trial : score 483.63563360175965, 
params {'learning_rate': 0.03321457011776751, 'n_estimators': 150, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}
5


[I 2022-12-17 08:03:41,552] Trial 7 finished with value: 685.8892945620081 and parameters: {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}. Best is trial 7 with value: 685.8892945620081.
[I 2022-12-17 08:03:41,574] Trial 8 finished with value: 562.4196560950369 and parameters: {'learning_rate': 0.029367583960700158, 'n_estimators': 118, 'colsample_bytree': 0.5152501249695247, 'subsample': 0.5186740943746072, 'reg_alpha': 4.113002803298292, 'reg_lambda': 1.8009532070563146}. Best is trial 7 with value: 685.8892945620081.
[I 2022-12-17 08:03:41,607] Trial 9 finished with value: 632.8183851395587 and parameters: {'learning_rate': 0.013578990752536595, 'n_estimators': 104, 'colsample_bytree': 0.8849967765493054, 'subsample': 0.6079105137484215, 'reg_alpha': 3.114452379095001, 'reg_lambda': 0.42673732496884}. Best is trial 7 with value: 685.8892

Best trial : score 685.8892945620081, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [15]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Seongdong_x, Seongdong_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [16]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Seongdong_x, Seongdong_y)):
    n_iter += 1
    print(n_iter)
    train_x= Seongdong_x.iloc[train_index]
    valid_x= Seongdong_x.iloc[test_index]
    train_y= Seongdong_y.iloc[train_index]
    valid_y= Seongdong_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

#for model Seongdong
#Best trial : score 382.6362715092011, 
#params {'learning_rate': 0.0015466895952366377, 'n_estimators': 163, 'colsample_bytree': 0.8534286719238086, 'subsample': 0.8645035840204937, 'reg_alpha': 3.8563517334297286, 'reg_lambda': 0.3702232586704518}

0it [00:00, ?it/s][I 2022-12-17 08:03:51,844] A new study created in memory with name: no-name-01987332-e835-49b9-baa2-5914227047df
[I 2022-12-17 08:03:51,898] Trial 0 finished with value: 296.29903830868886 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 296.29903830868886.
[I 2022-12-17 08:03:51,943] Trial 1 finished with value: 386.51533889666564 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 386.51533889666564.
[I 2022-12-17 08:03:51,965] Trial 2 finished with value: 331.68880519206886 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.59091248360

1


[I 2022-12-17 08:03:52,048] Trial 4 finished with value: 289.54482015405796 and parameters: {'learning_rate': 0.04615092843748656, 'n_estimators': 157, 'colsample_bytree': 0.5998368910791798, 'subsample': 0.7571172192068059, 'reg_alpha': 2.9620728443102124, 'reg_lambda': 0.23225206359998862}. Best is trial 1 with value: 386.51533889666564.
[I 2022-12-17 08:03:52,074] Trial 5 finished with value: 352.5363297603862 and parameters: {'learning_rate': 0.0611469403382424, 'n_estimators': 34, 'colsample_bytree': 0.5325257964926398, 'subsample': 0.9744427686266666, 'reg_alpha': 4.828160165372797, 'reg_lambda': 4.041986740582305}. Best is trial 1 with value: 386.51533889666564.
[I 2022-12-17 08:03:52,091] Trial 6 finished with value: 406.0951305577993 and parameters: {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}. Best is trial 6 with value: 406.0

Best trial : score 406.0951305577993, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:03:52,455] Trial 6 finished with value: 240.06040085553641 and parameters: {'learning_rate': 0.052750550108817414, 'n_estimators': 85, 'colsample_bytree': 0.5127095633720475, 'subsample': 0.5539457134966522, 'reg_alpha': 0.15714592843367126, 'reg_lambda': 3.182052056318902}. Best is trial 2 with value: 382.6362715092011.
[I 2022-12-17 08:03:52,491] Trial 7 finished with value: 258.4856340170139 and parameters: {'learning_rate': 0.03212124212655634, 'n_estimators': 102, 'colsample_bytree': 0.9537832369630466, 'subsample': 0.6246461145744375, 'reg_alpha': 2.0519146151781484, 'reg_lambda': 3.7777556927152434}. Best is trial 2 with value: 382.6362715092011.
[I 2022-12-17 08:03:52,509] Trial 8 finished with value: 368.98461214822566 and parameters: {'learning_rate': 0.023651018383670626, 'n_estimators': 15, 'colsample_bytree': 0.6448757264568841, 'subsample': 0.5806106436270022, 'reg_alpha': 4.648488261712865, 'reg_lambda': 4.040601897822085}. Best is trial 2 with value: 38

...

Best trial : score 434.8110523032481, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:03:53,138] Trial 5 finished with value: 297.3226424023191 and parameters: {'learning_rate': 0.05532464514729203, 'n_estimators': 139, 'colsample_bytree': 0.8259806297513003, 'subsample': 0.61213465473028, 'reg_alpha': 3.560896106737679, 'reg_lambda': 1.1862454374840004}. Best is trial 4 with value: 531.9622791397877.
[I 2022-12-17 08:03:53,183] Trial 6 finished with value: 301.4508993632184 and parameters: {'learning_rate': 0.03321457011776751, 'n_estimators': 150, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}. Best is trial 4 with value: 531.9622791397877.
[I 2022-12-17 08:03:53,210] Trial 7 finished with value: 438.5304251512997 and parameters: {'learning_rate': 0.010273802014981157, 'n_estimators': 73, 'colsample_bytree': 0.6326011838408627, 'subsample': 0.6219948216895418, 'reg_alpha': 4.865052773762228, 'reg_lambda': 1.965488623333802}. Best is trial 4 with value: 531.96227

Best trial : score 531.9622791397877, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:03:53,565] Trial 5 finished with value: 347.7532537289 and parameters: {'learning_rate': 0.08112675439237285, 'n_estimators': 162, 'colsample_bytree': 0.9335361592900518, 'subsample': 0.9566202762782356, 'reg_alpha': 2.556711994304689, 'reg_lambda': 2.507581473435998}. Best is trial 3 with value: 461.379488439401.
[I 2022-12-17 08:03:53,603] Trial 6 finished with value: 345.56896053054095 and parameters: {'learning_rate': 0.08003122271771075, 'n_estimators': 130, 'colsample_bytree': 0.8509834386288517, 'subsample': 0.8978963347180505, 'reg_alpha': 4.450026709087831, 'reg_lambda': 1.689975784257679}. Best is trial 3 with value: 461.379488439401.
[I 2022-12-17 08:03:53,620] Trial 7 finished with value: 498.93103534670877 and parameters: {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}. Best is trial 7 with value: 498.931035346

Best trial : score 498.93103534670877, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [17]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Gwangjin_x, Gwangjin_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 200}
LGBMRegressor(n_estimators=200)


In [18]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Gwangjin_x, Gwangjin_y)):
    n_iter += 1
    print(n_iter)
    train_x= Gwangjin_x.iloc[train_index]
    valid_x= Gwangjin_x.iloc[test_index]
    train_y= Gwangjin_y.iloc[train_index]
    valid_y= Gwangjin_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

#for model Gwangjin
#Best trial : score 232.5540590343182, 
#params {'learning_rate': 0.03818271231135446, 'n_estimators': 118, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}

0it [00:00, ?it/s][I 2022-12-17 08:04:00,953] A new study created in memory with name: no-name-e418d378-4082-422c-9601-14186262aa54
[I 2022-12-17 08:04:01,008] Trial 0 finished with value: 257.6491536663099 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 291, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 257.6491536663099.
[I 2022-12-17 08:04:01,058] Trial 1 finished with value: 266.23423596921543 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 274, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 266.23423596921543.
[I 2022-12-17 08:04:01,092] Trial 2 finished with value: 262.03092634266625 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 142, 'colsample_bytree': 0.590912483603

1


[I 2022-12-17 08:04:01,133] Trial 3 finished with value: 260.8556010219166 and parameters: {'learning_rate': 0.04376255684556946, 'n_estimators': 158, 'colsample_bytree': 0.8059264473611898, 'subsample': 0.569746930326021, 'reg_alpha': 1.4607232426760908, 'reg_lambda': 1.8318092164684585}. Best is trial 1 with value: 266.23423596921543.
[I 2022-12-17 08:04:01,181] Trial 4 finished with value: 262.2891449336554 and parameters: {'learning_rate': 0.04615092843748656, 'n_estimators': 257, 'colsample_bytree': 0.5998368910791798, 'subsample': 0.7571172192068059, 'reg_alpha': 2.9620728443102124, 'reg_lambda': 0.23225206359998862}. Best is trial 1 with value: 266.23423596921543.
[I 2022-12-17 08:04:01,213] Trial 5 finished with value: 263.0630943298311 and parameters: {'learning_rate': 0.0611469403382424, 'n_estimators': 134, 'colsample_bytree': 0.5325257964926398, 'subsample': 0.9744427686266666, 'reg_alpha': 4.828160165372797, 'reg_lambda': 4.041986740582305}. Best is trial 1 with value: 266

Best trial : score 267.4110029804463, 
params {'learning_rate': 0.05512431765498469, 'n_estimators': 137, 'colsample_bytree': 0.9847923138822793, 'subsample': 0.8875664116805573, 'reg_alpha': 4.697494707820946, 'reg_lambda': 4.474136752138244}
2


[I 2022-12-17 08:04:01,609] Trial 4 finished with value: 268.6340955140569 and parameters: {'learning_rate': 0.03178724984985056, 'n_estimators': 165, 'colsample_bytree': 0.864803089169032, 'subsample': 0.8187787356776066, 'reg_alpha': 4.436063712881633, 'reg_lambda': 2.3610746258097466}. Best is trial 2 with value: 368.00861733605194.
[I 2022-12-17 08:04:01,661] Trial 5 finished with value: 274.4191693911036 and parameters: {'learning_rate': 0.012839830347891868, 'n_estimators': 243, 'colsample_bytree': 0.8803925243084487, 'subsample': 0.7806385987847482, 'reg_alpha': 3.854835899772805, 'reg_lambda': 2.4689779818219537}. Best is trial 2 with value: 368.00861733605194.
[I 2022-12-17 08:04:01,708] Trial 6 finished with value: 264.64743488406367 and parameters: {'learning_rate': 0.052750550108817414, 'n_estimators': 185, 'colsample_bytree': 0.5127095633720475, 'subsample': 0.5539457134966522, 'reg_alpha': 0.15714592843367126, 'reg_lambda': 3.182052056318902}. Best is trial 2 with value: 

...

Best trial : score 291.40121633622175, 
params {'learning_rate': 0.005036739013921628, 'n_estimators': 218, 'colsample_bytree': 0.8387821809211412, 'subsample': 0.508293914463928, 'reg_alpha': 2.560465291496405, 'reg_lambda': 1.1324788759896898}
4


[I 2022-12-17 08:04:02,517] Trial 4 finished with value: 244.66247626869145 and parameters: {'learning_rate': 0.06103647690629941, 'n_estimators': 101, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}. Best is trial 4 with value: 244.66247626869145.
[I 2022-12-17 08:04:02,562] Trial 5 finished with value: 229.5995697717774 and parameters: {'learning_rate': 0.05532464514729203, 'n_estimators': 239, 'colsample_bytree': 0.8259806297513003, 'subsample': 0.61213465473028, 'reg_alpha': 3.560896106737679, 'reg_lambda': 1.1862454374840004}. Best is trial 4 with value: 244.66247626869145.
[I 2022-12-17 08:04:02,619] Trial 6 finished with value: 226.7120756827267 and parameters: {'learning_rate': 0.03321457011776751, 'n_estimators': 250, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}. Best is trial 4 with value: 24

Best trial : score 256.93668159914057, 
params {'learning_rate': 0.010273802014981157, 'n_estimators': 173, 'colsample_bytree': 0.6326011838408627, 'subsample': 0.6219948216895418, 'reg_alpha': 4.865052773762228, 'reg_lambda': 1.965488623333802}
5


[I 2022-12-17 08:04:03,002] Trial 4 finished with value: 218.88579429866846 and parameters: {'learning_rate': 0.0878599341208676, 'n_estimators': 248, 'colsample_bytree': 0.848507870497634, 'subsample': 0.8512420419935547, 'reg_alpha': 1.7974557560987758, 'reg_lambda': 1.4679592213224668}. Best is trial 2 with value: 225.3078574001858.
[I 2022-12-17 08:04:03,055] Trial 5 finished with value: 218.82053639763035 and parameters: {'learning_rate': 0.08112675439237285, 'n_estimators': 262, 'colsample_bytree': 0.9335361592900518, 'subsample': 0.9566202762782356, 'reg_alpha': 2.556711994304689, 'reg_lambda': 2.507581473435998}. Best is trial 2 with value: 225.3078574001858.
[I 2022-12-17 08:04:03,101] Trial 6 finished with value: 217.597587158715 and parameters: {'learning_rate': 0.08003122271771075, 'n_estimators': 230, 'colsample_bytree': 0.8509834386288517, 'subsample': 0.8978963347180505, 'reg_alpha': 4.450026709087831, 'reg_lambda': 1.689975784257679}. Best is trial 2 with value: 225.307

Best trial : score 232.5540590343182, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 118, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [19]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Dongdaemun_x, Dongdaemun_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [20]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Dongdaemun_x, Dongdaemun_y)):
    n_iter += 1
    print(n_iter)
    train_x= Dongdaemun_x.iloc[train_index]
    valid_x= Dongdaemun_x.iloc[test_index]
    train_y= Dongdaemun_y.iloc[train_index]
    valid_y= Dongdaemun_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

#for model Dongdaemun
#Best trial : score 240.13905925712606, 
#params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}

0it [00:00, ?it/s][I 2022-12-17 08:04:17,240] A new study created in memory with name: no-name-8b6f7ca6-7483-472e-97c8-b46259eff6ae
[I 2022-12-17 08:04:17,302] Trial 0 finished with value: 207.75391907436133 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 207.75391907436133.
[I 2022-12-17 08:04:17,358] Trial 1 finished with value: 232.03936709809594 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 232.03936709809594.
[I 2022-12-17 08:04:17,381] Trial 2 finished with value: 212.31792053418468 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.59091248360

1


[I 2022-12-17 08:04:17,414] Trial 3 finished with value: 213.24835777033147 and parameters: {'learning_rate': 0.04376255684556946, 'n_estimators': 58, 'colsample_bytree': 0.8059264473611898, 'subsample': 0.569746930326021, 'reg_alpha': 1.4607232426760908, 'reg_lambda': 1.8318092164684585}. Best is trial 1 with value: 232.03936709809594.
[I 2022-12-17 08:04:17,466] Trial 4 finished with value: 207.4561073800705 and parameters: {'learning_rate': 0.04615092843748656, 'n_estimators': 157, 'colsample_bytree': 0.5998368910791798, 'subsample': 0.7571172192068059, 'reg_alpha': 2.9620728443102124, 'reg_lambda': 0.23225206359998862}. Best is trial 1 with value: 232.03936709809594.
[I 2022-12-17 08:04:17,489] Trial 5 finished with value: 217.78397745320063 and parameters: {'learning_rate': 0.0611469403382424, 'n_estimators': 34, 'colsample_bytree': 0.5325257964926398, 'subsample': 0.9744427686266666, 'reg_alpha': 4.828160165372797, 'reg_lambda': 4.041986740582305}. Best is trial 1 with value: 232

Best trial : score 245.15898561152244, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:04:17,935] Trial 5 finished with value: 198.15875240573902 and parameters: {'learning_rate': 0.012839830347891868, 'n_estimators': 143, 'colsample_bytree': 0.8803925243084487, 'subsample': 0.7806385987847482, 'reg_alpha': 3.854835899772805, 'reg_lambda': 2.4689779818219537}. Best is trial 2 with value: 253.8315747398908.
[I 2022-12-17 08:04:17,985] Trial 6 finished with value: 188.22687027312955 and parameters: {'learning_rate': 0.052750550108817414, 'n_estimators': 85, 'colsample_bytree': 0.5127095633720475, 'subsample': 0.5539457134966522, 'reg_alpha': 0.15714592843367126, 'reg_lambda': 3.182052056318902}. Best is trial 2 with value: 253.8315747398908.
[I 2022-12-17 08:04:18,025] Trial 7 finished with value: 182.87951329545325 and parameters: {'learning_rate': 0.03212124212655634, 'n_estimators': 102, 'colsample_bytree': 0.9537832369630466, 'subsample': 0.6246461145744375, 'reg_alpha': 2.0519146151781484, 'reg_lambda': 3.7777556927152434}. Best is trial 2 with value:

...

Best trial : score 299.2317944515763, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:04:18,795] Trial 5 finished with value: 165.06320514009806 and parameters: {'learning_rate': 0.05532464514729203, 'n_estimators': 139, 'colsample_bytree': 0.8259806297513003, 'subsample': 0.61213465473028, 'reg_alpha': 3.560896106737679, 'reg_lambda': 1.1862454374840004}. Best is trial 4 with value: 255.1030390895179.
[I 2022-12-17 08:04:18,850] Trial 6 finished with value: 163.85159750834038 and parameters: {'learning_rate': 0.03321457011776751, 'n_estimators': 150, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}. Best is trial 4 with value: 255.1030390895179.
[I 2022-12-17 08:04:18,881] Trial 7 finished with value: 215.50064021743748 and parameters: {'learning_rate': 0.010273802014981157, 'n_estimators': 73, 'colsample_bytree': 0.6326011838408627, 'subsample': 0.6219948216895418, 'reg_alpha': 4.865052773762228, 'reg_lambda': 1.965488623333802}. Best is trial 4 with value: 255.10

Best trial : score 255.1030390895179, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:04:19,241] Trial 4 finished with value: 150.51381446534316 and parameters: {'learning_rate': 0.0878599341208676, 'n_estimators': 148, 'colsample_bytree': 0.848507870497634, 'subsample': 0.8512420419935547, 'reg_alpha': 1.7974557560987758, 'reg_lambda': 1.4679592213224668}. Best is trial 3 with value: 220.93230314196404.
[I 2022-12-17 08:04:19,296] Trial 5 finished with value: 150.45692182727905 and parameters: {'learning_rate': 0.08112675439237285, 'n_estimators': 162, 'colsample_bytree': 0.9335361592900518, 'subsample': 0.9566202762782356, 'reg_alpha': 2.556711994304689, 'reg_lambda': 2.507581473435998}. Best is trial 3 with value: 220.93230314196404.
[I 2022-12-17 08:04:19,352] Trial 6 finished with value: 155.2688884415333 and parameters: {'learning_rate': 0.08003122271771075, 'n_estimators': 130, 'colsample_bytree': 0.8509834386288517, 'subsample': 0.8978963347180505, 'reg_alpha': 4.450026709087831, 'reg_lambda': 1.689975784257679}. Best is trial 3 with value: 220.

Best trial : score 240.13905925712606, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [21]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Jungnang_x, Jungnang_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 300}
LGBMRegressor(n_estimators=300)


In [22]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 200, 400),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Jungnang_x, Jungnang_y)):
    n_iter += 1
    print(n_iter)
    train_x= Jungnang_x.iloc[train_index]
    valid_x= Jungnang_x.iloc[test_index]
    train_y= Jungnang_y.iloc[train_index]
    valid_y= Jungnang_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

#for model Jungnang
#Best trial : score 122.1451221451664, 
#params {'learning_rate': 0.013578990752536595, 'n_estimators': 304, 'colsample_bytree': 0.8849967765493054, 'subsample': 0.6079105137484215, 'reg_alpha': 3.114452379095001, 'reg_lambda': 0.42673732496884}

0it [00:00, ?it/s][I 2022-12-17 08:04:34,809] A new study created in memory with name: no-name-8a85ddcf-5bd5-4c7b-b31a-3c869c517b8c
[I 2022-12-17 08:04:34,929] Trial 0 finished with value: 102.04314496440392 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 391, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 102.04314496440392.


1


[I 2022-12-17 08:04:35,042] Trial 1 finished with value: 126.1438407130506 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 374, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 126.1438407130506.
[I 2022-12-17 08:04:35,113] Trial 2 finished with value: 104.85261729941453 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 242, 'colsample_bytree': 0.5909124836035503, 'subsample': 0.5917022549267169, 'reg_alpha': 1.5212112147976886, 'reg_lambda': 2.6237821581611893}. Best is trial 1 with value: 126.1438407130506.
[I 2022-12-17 08:04:35,194] Trial 3 finished with value: 103.66809672774396 and parameters: {'learning_rate': 0.04376255684556946, 'n_estimators': 258, 'colsample_bytree': 0.8059264473611898, 'subsample': 0.569746930326021, 'reg_alpha': 1.4607232426760908, 'reg_lambda': 1.8318092164684585}. Best is trial 1 with value:

Best trial : score 137.93457708236178, 
params {'learning_rate': 0.004404463590406622, 'n_estimators': 382, 'colsample_bytree': 0.6293899908000085, 'subsample': 0.831261142176991, 'reg_alpha': 1.5585553804470549, 'reg_lambda': 2.600340105889054}
2


[I 2022-12-17 08:04:36,042] Trial 2 finished with value: 166.43646016400814 and parameters: {'learning_rate': 0.0015466895952366377, 'n_estimators': 363, 'colsample_bytree': 0.8534286719238086, 'subsample': 0.8645035840204937, 'reg_alpha': 3.8563517334297286, 'reg_lambda': 0.3702232586704518}. Best is trial 2 with value: 166.43646016400814.
[I 2022-12-17 08:04:36,122] Trial 3 finished with value: 116.70917960198284 and parameters: {'learning_rate': 0.03648810712588299, 'n_estimators': 223, 'colsample_bytree': 0.9315517129377968, 'subsample': 0.811649063413779, 'reg_alpha': 1.654490124263246, 'reg_lambda': 0.3177917514301182}. Best is trial 2 with value: 166.43646016400814.
[I 2022-12-17 08:04:36,205] Trial 4 finished with value: 115.07062406144635 and parameters: {'learning_rate': 0.03178724984985056, 'n_estimators': 265, 'colsample_bytree': 0.864803089169032, 'subsample': 0.8187787356776066, 'reg_alpha': 4.436063712881633, 'reg_lambda': 2.3610746258097466}. Best is trial 2 with value:

...

Best trial : score 125.60848359766561, 
params {'learning_rate': 0.005036739013921628, 'n_estimators': 318, 'colsample_bytree': 0.8387821809211412, 'subsample': 0.508293914463928, 'reg_alpha': 2.560465291496405, 'reg_lambda': 1.1324788759896898}
4


[I 2022-12-17 08:04:37,973] Trial 2 finished with value: 126.31938791545412 and parameters: {'learning_rate': 0.09014138765916972, 'n_estimators': 327, 'colsample_bytree': 0.6695148955243504, 'subsample': 0.6746047873063304, 'reg_alpha': 3.629778394351197, 'reg_lambda': 4.485551299762886}. Best is trial 0 with value: 134.99440726130618.
[I 2022-12-17 08:04:38,089] Trial 3 finished with value: 125.7444194938123 and parameters: {'learning_rate': 0.08882155600224662, 'n_estimators': 356, 'colsample_bytree': 0.8210158230771438, 'subsample': 0.5420699824975244, 'reg_alpha': 0.8081435704730688, 'reg_lambda': 4.492770942635397}. Best is trial 0 with value: 134.99440726130618.
[I 2022-12-17 08:04:38,149] Trial 4 finished with value: 135.90305729489722 and parameters: {'learning_rate': 0.06103647690629941, 'n_estimators': 201, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}. Best is trial 4 with value:

Best trial : score 148.73907685784326, 
params {'learning_rate': 0.010273802014981157, 'n_estimators': 273, 'colsample_bytree': 0.6326011838408627, 'subsample': 0.6219948216895418, 'reg_alpha': 4.865052773762228, 'reg_lambda': 1.965488623333802}
5


[I 2022-12-17 08:04:38,916] Trial 1 finished with value: 112.98517166167558 and parameters: {'learning_rate': 0.04339023068341412, 'n_estimators': 394, 'colsample_bytree': 0.9818099885446264, 'subsample': 0.92650472773368, 'reg_alpha': 1.4722444603479286, 'reg_lambda': 1.9254886430096263}. Best is trial 0 with value: 114.34653518117804.
[I 2022-12-17 08:04:38,990] Trial 2 finished with value: 112.34184512825546 and parameters: {'learning_rate': 0.08526253048016884, 'n_estimators': 263, 'colsample_bytree': 0.5847463733430462, 'subsample': 0.7784006312291751, 'reg_alpha': 4.680773870803905, 'reg_lambda': 3.480148983374865}. Best is trial 0 with value: 114.34653518117804.
[I 2022-12-17 08:04:39,091] Trial 3 finished with value: 112.16479515715727 and parameters: {'learning_rate': 0.05743605583884714, 'n_estimators': 219, 'colsample_bytree': 0.8075036133495849, 'subsample': 0.9950269250521316, 'reg_alpha': 0.7004200761826201, 'reg_lambda': 2.591648261818684}. Best is trial 0 with value: 11

Best trial : score 122.1451221451664, 
params {'learning_rate': 0.013578990752536595, 'n_estimators': 304, 'colsample_bytree': 0.8849967765493054, 'subsample': 0.6079105137484215, 'reg_alpha': 3.114452379095001, 'reg_lambda': 0.42673732496884}


In [23]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Seongbuk_x, Seongbuk_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [24]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Seongbuk_x, Seongbuk_y)):
    n_iter += 1
    print(n_iter)
    train_x= Seongbuk_x.iloc[train_index]
    valid_x= Seongbuk_x.iloc[test_index]
    train_y= Seongbuk_y.iloc[train_index]
    valid_y= Seongbuk_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

#for model Seongbuk
#Best trial : score 195.90765638075416, 
#params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}

0it [00:00, ?it/s][I 2022-12-17 08:04:50,323] A new study created in memory with name: no-name-c2dc6fa8-4f20-4ba8-8047-8bd11bc6e31a
[I 2022-12-17 08:04:50,374] Trial 0 finished with value: 164.6497024075988 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 164.6497024075988.
[I 2022-12-17 08:04:50,421] Trial 1 finished with value: 179.94107015745155 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 179.94107015745155.
[I 2022-12-17 08:04:50,443] Trial 2 finished with value: 175.43346273459653 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.5909124836035

1


[I 2022-12-17 08:04:50,518] Trial 4 finished with value: 172.40612241531585 and parameters: {'learning_rate': 0.04615092843748656, 'n_estimators': 157, 'colsample_bytree': 0.5998368910791798, 'subsample': 0.7571172192068059, 'reg_alpha': 2.9620728443102124, 'reg_lambda': 0.23225206359998862}. Best is trial 1 with value: 179.94107015745155.
[I 2022-12-17 08:04:50,538] Trial 5 finished with value: 179.56341519240962 and parameters: {'learning_rate': 0.0611469403382424, 'n_estimators': 34, 'colsample_bytree': 0.5325257964926398, 'subsample': 0.9744427686266666, 'reg_alpha': 4.828160165372797, 'reg_lambda': 4.041986740582305}. Best is trial 1 with value: 179.94107015745155.
[I 2022-12-17 08:04:50,554] Trial 6 finished with value: 195.90765638075416 and parameters: {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}. Best is trial 6 with value: 195

Best trial : score 195.90765638075416, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:04:50,913] Trial 6 finished with value: 136.98073852602863 and parameters: {'learning_rate': 0.052750550108817414, 'n_estimators': 85, 'colsample_bytree': 0.5127095633720475, 'subsample': 0.5539457134966522, 'reg_alpha': 0.15714592843367126, 'reg_lambda': 3.182052056318902}. Best is trial 2 with value: 227.3511059104103.
[I 2022-12-17 08:04:50,960] Trial 7 finished with value: 137.6434752657819 and parameters: {'learning_rate': 0.03212124212655634, 'n_estimators': 102, 'colsample_bytree': 0.9537832369630466, 'subsample': 0.6246461145744375, 'reg_alpha': 2.0519146151781484, 'reg_lambda': 3.7777556927152434}. Best is trial 2 with value: 227.3511059104103.
[I 2022-12-17 08:04:50,979] Trial 8 finished with value: 223.889821515482 and parameters: {'learning_rate': 0.023651018383670626, 'n_estimators': 15, 'colsample_bytree': 0.6448757264568841, 'subsample': 0.5806106436270022, 'reg_alpha': 4.648488261712865, 'reg_lambda': 4.040601897822085}. Best is trial 2 with value: 227.

...

Best trial : score 204.29487537765468, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:04:51,660] Trial 5 finished with value: 162.139245269535 and parameters: {'learning_rate': 0.05532464514729203, 'n_estimators': 139, 'colsample_bytree': 0.8259806297513003, 'subsample': 0.61213465473028, 'reg_alpha': 3.560896106737679, 'reg_lambda': 1.1862454374840004}. Best is trial 4 with value: 242.87254604100224.
[I 2022-12-17 08:04:51,710] Trial 6 finished with value: 168.66464963240117 and parameters: {'learning_rate': 0.03321457011776751, 'n_estimators': 150, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}. Best is trial 4 with value: 242.87254604100224.
[I 2022-12-17 08:04:51,737] Trial 7 finished with value: 216.06784896306314 and parameters: {'learning_rate': 0.010273802014981157, 'n_estimators': 73, 'colsample_bytree': 0.6326011838408627, 'subsample': 0.6219948216895418, 'reg_alpha': 4.865052773762228, 'reg_lambda': 1.965488623333802}. Best is trial 4 with value: 242.87

Best trial : score 242.87254604100224, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:04:52,063] Trial 4 finished with value: 177.16211116537372 and parameters: {'learning_rate': 0.0878599341208676, 'n_estimators': 148, 'colsample_bytree': 0.848507870497634, 'subsample': 0.8512420419935547, 'reg_alpha': 1.7974557560987758, 'reg_lambda': 1.4679592213224668}. Best is trial 3 with value: 196.7632617705352.
[I 2022-12-17 08:04:52,111] Trial 5 finished with value: 174.7666959269692 and parameters: {'learning_rate': 0.08112675439237285, 'n_estimators': 162, 'colsample_bytree': 0.9335361592900518, 'subsample': 0.9566202762782356, 'reg_alpha': 2.556711994304689, 'reg_lambda': 2.507581473435998}. Best is trial 3 with value: 196.7632617705352.
[I 2022-12-17 08:04:52,151] Trial 6 finished with value: 175.9113496320829 and parameters: {'learning_rate': 0.08003122271771075, 'n_estimators': 130, 'colsample_bytree': 0.8509834386288517, 'subsample': 0.8978963347180505, 'reg_alpha': 4.450026709087831, 'reg_lambda': 1.689975784257679}. Best is trial 3 with value: 196.763

Best trial : score 213.50807623321114, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [25]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Gangbuk_x, Gangbuk_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [26]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Gangbuk_x, Gangbuk_y)):
    n_iter += 1
    print(n_iter)
    train_x= Gangbuk_x.iloc[train_index]
    valid_x= Gangbuk_x.iloc[test_index]
    train_y= Gangbuk_y.iloc[train_index]
    valid_y= Gangbuk_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

#for model Gangbuk
#Best trial : score 186.07814990133724, 
#params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}

0it [00:00, ?it/s][I 2022-12-17 08:04:58,799] A new study created in memory with name: no-name-ebc1ffbb-e0a0-4004-9a30-59335b0f8d5a
[I 2022-12-17 08:04:58,838] Trial 0 finished with value: 172.09839864605544 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 172.09839864605544.
[I 2022-12-17 08:04:58,872] Trial 1 finished with value: 199.19807195194062 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 199.19807195194062.
[I 2022-12-17 08:04:58,890] Trial 2 finished with value: 184.2096179591493 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.590912483603

1


[I 2022-12-17 08:04:58,979] Trial 6 finished with value: 202.84163342861928 and parameters: {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}. Best is trial 6 with value: 202.84163342861928.
[I 2022-12-17 08:04:59,017] Trial 7 finished with value: 199.18142030272844 and parameters: {'learning_rate': 0.004404463590406622, 'n_estimators': 182, 'colsample_bytree': 0.6293899908000085, 'subsample': 0.831261142176991, 'reg_alpha': 1.5585553804470549, 'reg_lambda': 2.600340105889054}. Best is trial 6 with value: 202.84163342861928.
[I 2022-12-17 08:04:59,033] Trial 8 finished with value: 195.23653530661744 and parameters: {'learning_rate': 0.05512431765498469, 'n_estimators': 37, 'colsample_bytree': 0.9847923138822793, 'subsample': 0.8875664116805573, 'reg_alpha': 4.697494707820946, 'reg_lambda': 4.474136752138244}. Best is trial 6 with value: 202.

Best trial : score 202.84163342861928, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:04:59,273] Trial 6 finished with value: 216.94747661124396 and parameters: {'learning_rate': 0.052750550108817414, 'n_estimators': 85, 'colsample_bytree': 0.5127095633720475, 'subsample': 0.5539457134966522, 'reg_alpha': 0.15714592843367126, 'reg_lambda': 3.182052056318902}. Best is trial 2 with value: 242.5264838724763.
[I 2022-12-17 08:04:59,303] Trial 7 finished with value: 218.88394316838023 and parameters: {'learning_rate': 0.03212124212655634, 'n_estimators': 102, 'colsample_bytree': 0.9537832369630466, 'subsample': 0.6246461145744375, 'reg_alpha': 2.0519146151781484, 'reg_lambda': 3.7777556927152434}. Best is trial 2 with value: 242.5264838724763.
[I 2022-12-17 08:04:59,317] Trial 8 finished with value: 241.5420375953559 and parameters: {'learning_rate': 0.023651018383670626, 'n_estimators': 15, 'colsample_bytree': 0.6448757264568841, 'subsample': 0.5806106436270022, 'reg_alpha': 4.648488261712865, 'reg_lambda': 4.040601897822085}. Best is trial 2 with value: 24

...

Best trial : score 186.07814990133724, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:04:59,892] Trial 6 finished with value: 164.3504741406183 and parameters: {'learning_rate': 0.03321457011776751, 'n_estimators': 150, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}. Best is trial 4 with value: 204.5004573191171.
[I 2022-12-17 08:04:59,916] Trial 7 finished with value: 186.18410611607928 and parameters: {'learning_rate': 0.010273802014981157, 'n_estimators': 73, 'colsample_bytree': 0.6326011838408627, 'subsample': 0.6219948216895418, 'reg_alpha': 4.865052773762228, 'reg_lambda': 1.965488623333802}. Best is trial 4 with value: 204.5004573191171.
[I 2022-12-17 08:04:59,947] Trial 8 finished with value: 166.77693192337665 and parameters: {'learning_rate': 0.08931260896253422, 'n_estimators': 126, 'colsample_bytree': 0.8974056517708242, 'subsample': 0.751318546552596, 'reg_alpha': 2.8845194231317954, 'reg_lambda': 2.4625884690943196}. Best is trial 4 with value: 204.5

Best trial : score 204.5004573191171, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:05:00,201] Trial 5 finished with value: 187.63132991225262 and parameters: {'learning_rate': 0.08112675439237285, 'n_estimators': 162, 'colsample_bytree': 0.9335361592900518, 'subsample': 0.9566202762782356, 'reg_alpha': 2.556711994304689, 'reg_lambda': 2.507581473435998}. Best is trial 0 with value: 192.76856460583332.
[I 2022-12-17 08:05:00,250] Trial 6 finished with value: 184.65785454184254 and parameters: {'learning_rate': 0.08003122271771075, 'n_estimators': 130, 'colsample_bytree': 0.8509834386288517, 'subsample': 0.8978963347180505, 'reg_alpha': 4.450026709087831, 'reg_lambda': 1.689975784257679}. Best is trial 0 with value: 192.76856460583332.
[I 2022-12-17 08:05:00,268] Trial 7 finished with value: 152.004370072926 and parameters: {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}. Best is trial 0 with value: 192.768

Best trial : score 192.76856460583332, 
params {'learning_rate': 0.09410539985093852, 'n_estimators': 191, 'colsample_bytree': 0.9574321951102243, 'subsample': 0.6850793501277221, 'reg_alpha': 0.07728308264433714, 'reg_lambda': 4.641592812938627}


In [27]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Dobong_x, Dobong_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [28]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Dobong_x, Dobong_y)):
    n_iter += 1
    print(n_iter)
    train_x= Dobong_x.iloc[train_index]
    valid_x= Dobong_x.iloc[test_index]
    train_y= Dobong_y.iloc[train_index]
    valid_y= Dobong_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

#for model Dobong
#Best trial : score 175.02849182476825, 
#params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}

0it [00:00, ?it/s][I 2022-12-17 08:05:14,408] A new study created in memory with name: no-name-9e9e9431-d9b9-4165-8bdd-45fe879fbc2b
[I 2022-12-17 08:05:14,461] Trial 0 finished with value: 124.94711475596398 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 124.94711475596398.
[I 2022-12-17 08:05:14,507] Trial 1 finished with value: 154.3683140134212 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 154.3683140134212.
[I 2022-12-17 08:05:14,528] Trial 2 finished with value: 131.85307840666349 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.5909124836035

1


[I 2022-12-17 08:05:14,599] Trial 4 finished with value: 129.92000190400512 and parameters: {'learning_rate': 0.04615092843748656, 'n_estimators': 157, 'colsample_bytree': 0.5998368910791798, 'subsample': 0.7571172192068059, 'reg_alpha': 2.9620728443102124, 'reg_lambda': 0.23225206359998862}. Best is trial 1 with value: 154.3683140134212.
[I 2022-12-17 08:05:14,623] Trial 5 finished with value: 139.61474627326282 and parameters: {'learning_rate': 0.0611469403382424, 'n_estimators': 34, 'colsample_bytree': 0.5325257964926398, 'subsample': 0.9744427686266666, 'reg_alpha': 4.828160165372797, 'reg_lambda': 4.041986740582305}. Best is trial 1 with value: 154.3683140134212.
[I 2022-12-17 08:05:14,648] Trial 6 finished with value: 175.02849182476825 and parameters: {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}. Best is trial 6 with value: 175.0

Best trial : score 175.02849182476825, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:05:15,024] Trial 5 finished with value: 145.95346335680355 and parameters: {'learning_rate': 0.012839830347891868, 'n_estimators': 143, 'colsample_bytree': 0.8803925243084487, 'subsample': 0.7806385987847482, 'reg_alpha': 3.854835899772805, 'reg_lambda': 2.4689779818219537}. Best is trial 2 with value: 209.06587388057278.
[I 2022-12-17 08:05:15,054] Trial 6 finished with value: 132.18232275223357 and parameters: {'learning_rate': 0.052750550108817414, 'n_estimators': 85, 'colsample_bytree': 0.5127095633720475, 'subsample': 0.5539457134966522, 'reg_alpha': 0.15714592843367126, 'reg_lambda': 3.182052056318902}. Best is trial 2 with value: 209.06587388057278.
[I 2022-12-17 08:05:15,091] Trial 7 finished with value: 138.0741163950171 and parameters: {'learning_rate': 0.03212124212655634, 'n_estimators': 102, 'colsample_bytree': 0.9537832369630466, 'subsample': 0.6246461145744375, 'reg_alpha': 2.0519146151781484, 'reg_lambda': 3.7777556927152434}. Best is trial 2 with value

...

Best trial : score 189.40810733878007, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:05:15,796] Trial 5 finished with value: 112.81612942652049 and parameters: {'learning_rate': 0.05532464514729203, 'n_estimators': 139, 'colsample_bytree': 0.8259806297513003, 'subsample': 0.61213465473028, 'reg_alpha': 3.560896106737679, 'reg_lambda': 1.1862454374840004}. Best is trial 4 with value: 181.94622017319406.
[I 2022-12-17 08:05:15,857] Trial 6 finished with value: 112.90746469849786 and parameters: {'learning_rate': 0.03321457011776751, 'n_estimators': 150, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}. Best is trial 4 with value: 181.94622017319406.
[I 2022-12-17 08:05:15,891] Trial 7 finished with value: 140.18108065163304 and parameters: {'learning_rate': 0.010273802014981157, 'n_estimators': 73, 'colsample_bytree': 0.6326011838408627, 'subsample': 0.6219948216895418, 'reg_alpha': 4.865052773762228, 'reg_lambda': 1.965488623333802}. Best is trial 4 with value: 181.

Best trial : score 181.94622017319406, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:05:16,221] Trial 4 finished with value: 148.95256555142385 and parameters: {'learning_rate': 0.0878599341208676, 'n_estimators': 148, 'colsample_bytree': 0.848507870497634, 'subsample': 0.8512420419935547, 'reg_alpha': 1.7974557560987758, 'reg_lambda': 1.4679592213224668}. Best is trial 3 with value: 174.96062424901035.
[I 2022-12-17 08:05:16,270] Trial 5 finished with value: 153.43030361871962 and parameters: {'learning_rate': 0.08112675439237285, 'n_estimators': 162, 'colsample_bytree': 0.9335361592900518, 'subsample': 0.9566202762782356, 'reg_alpha': 2.556711994304689, 'reg_lambda': 2.507581473435998}. Best is trial 3 with value: 174.96062424901035.
[I 2022-12-17 08:05:16,312] Trial 6 finished with value: 151.03660036422207 and parameters: {'learning_rate': 0.08003122271771075, 'n_estimators': 130, 'colsample_bytree': 0.8509834386288517, 'subsample': 0.8978963347180505, 'reg_alpha': 4.450026709087831, 'reg_lambda': 1.689975784257679}. Best is trial 3 with value: 174

Best trial : score 181.53176945909797, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [29]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Nowon_x, Nowon_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [30]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Nowon_x, Nowon_y)):
    n_iter += 1
    print(n_iter)
    train_x= Nowon_x.iloc[train_index]
    valid_x= Nowon_x.iloc[test_index]
    train_y= Nowon_y.iloc[train_index]
    valid_y= Nowon_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

#for model Nowon
#Best trial : score 158.37180515096702, 
#params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}

0it [00:00, ?it/s][I 2022-12-17 08:05:42,302] A new study created in memory with name: no-name-29afa2b7-cf43-4a12-b38c-3b8f6b9bd4a0
[I 2022-12-17 08:05:42,392] Trial 0 finished with value: 138.53342040286446 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 138.53342040286446.
[I 2022-12-17 08:05:42,466] Trial 1 finished with value: 159.47004334753834 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 159.47004334753834.


1


[I 2022-12-17 08:05:42,497] Trial 2 finished with value: 146.25969687645446 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.5909124836035503, 'subsample': 0.5917022549267169, 'reg_alpha': 1.5212112147976886, 'reg_lambda': 2.6237821581611893}. Best is trial 1 with value: 159.47004334753834.
[I 2022-12-17 08:05:42,534] Trial 3 finished with value: 146.22570957473025 and parameters: {'learning_rate': 0.04376255684556946, 'n_estimators': 58, 'colsample_bytree': 0.8059264473611898, 'subsample': 0.569746930326021, 'reg_alpha': 1.4607232426760908, 'reg_lambda': 1.8318092164684585}. Best is trial 1 with value: 159.47004334753834.
[I 2022-12-17 08:05:42,603] Trial 4 finished with value: 138.77865953307537 and parameters: {'learning_rate': 0.04615092843748656, 'n_estimators': 157, 'colsample_bytree': 0.5998368910791798, 'subsample': 0.7571172192068059, 'reg_alpha': 2.9620728443102124, 'reg_lambda': 0.23225206359998862}. Best is trial 1 with value:

Best trial : score 171.7584746350173, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:05:43,087] Trial 3 finished with value: 158.1185604566902 and parameters: {'learning_rate': 0.03648810712588299, 'n_estimators': 23, 'colsample_bytree': 0.9315517129377968, 'subsample': 0.811649063413779, 'reg_alpha': 1.654490124263246, 'reg_lambda': 0.3177917514301182}. Best is trial 2 with value: 184.39520614067393.
[I 2022-12-17 08:05:43,137] Trial 4 finished with value: 143.10931630181864 and parameters: {'learning_rate': 0.03178724984985056, 'n_estimators': 65, 'colsample_bytree': 0.864803089169032, 'subsample': 0.8187787356776066, 'reg_alpha': 4.436063712881633, 'reg_lambda': 2.3610746258097466}. Best is trial 2 with value: 184.39520614067393.
[I 2022-12-17 08:05:43,206] Trial 5 finished with value: 144.54913136090477 and parameters: {'learning_rate': 0.012839830347891868, 'n_estimators': 143, 'colsample_bytree': 0.8803925243084487, 'subsample': 0.7806385987847482, 'reg_alpha': 3.854835899772805, 'reg_lambda': 2.4689779818219537}. Best is trial 2 with value: 184.

...

Best trial : score 202.33070897125197, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:05:44,284] Trial 3 finished with value: 146.79644134622208 and parameters: {'learning_rate': 0.08882155600224662, 'n_estimators': 156, 'colsample_bytree': 0.8210158230771438, 'subsample': 0.5420699824975244, 'reg_alpha': 0.8081435704730688, 'reg_lambda': 4.492770942635397}. Best is trial 0 with value: 174.7887990324796.
[I 2022-12-17 08:05:44,299] Trial 4 finished with value: 206.1926082684721 and parameters: {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}. Best is trial 4 with value: 206.1926082684721.
[I 2022-12-17 08:05:44,367] Trial 5 finished with value: 145.61663321181985 and parameters: {'learning_rate': 0.05532464514729203, 'n_estimators': 139, 'colsample_bytree': 0.8259806297513003, 'subsample': 0.61213465473028, 'reg_alpha': 3.560896106737679, 'reg_lambda': 1.1862454374840004}. Best is trial 4 with value: 206.

Best trial : score 206.1926082684721, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:05:44,882] Trial 1 finished with value: 127.26378403757893 and parameters: {'learning_rate': 0.04339023068341412, 'n_estimators': 194, 'colsample_bytree': 0.9818099885446264, 'subsample': 0.92650472773368, 'reg_alpha': 1.4722444603479286, 'reg_lambda': 1.9254886430096263}. Best is trial 0 with value: 127.79644964931924.
[I 2022-12-17 08:05:44,919] Trial 2 finished with value: 131.17962930203794 and parameters: {'learning_rate': 0.08526253048016884, 'n_estimators': 63, 'colsample_bytree': 0.5847463733430462, 'subsample': 0.7784006312291751, 'reg_alpha': 4.680773870803905, 'reg_lambda': 3.480148983374865}. Best is trial 2 with value: 131.17962930203794.
[I 2022-12-17 08:05:44,944] Trial 3 finished with value: 148.03979166086447 and parameters: {'learning_rate': 0.05743605583884714, 'n_estimators': 19, 'colsample_bytree': 0.8075036133495849, 'subsample': 0.9950269250521316, 'reg_alpha': 0.7004200761826201, 'reg_lambda': 2.591648261818684}. Best is trial 3 with value: 148.

Best trial : score 158.37180515096702, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [31]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Eunpyeong_x, Eunpyeong_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [32]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Eunpyeong_x, Eunpyeong_y)):
    n_iter += 1
    print(n_iter)
    train_x= Eunpyeong_x.iloc[train_index]
    valid_x= Eunpyeong_x.iloc[test_index]
    train_y= Eunpyeong_y.iloc[train_index]
    valid_y= Eunpyeong_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

#for model Eunpyeong
#Best trial : score 189.6534130865682, 
#params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}

0it [00:00, ?it/s][I 2022-12-17 08:06:04,009] A new study created in memory with name: no-name-43426c48-d1d1-4180-b1b7-e4f3ccf2cc7e
[I 2022-12-17 08:06:04,093] Trial 0 finished with value: 122.4697114173747 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 122.4697114173747.
[I 2022-12-17 08:06:04,149] Trial 1 finished with value: 173.9959086138952 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 173.9959086138952.
[I 2022-12-17 08:06:04,174] Trial 2 finished with value: 150.27808852476525 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.590912483603550

1


[I 2022-12-17 08:06:04,212] Trial 3 finished with value: 139.16472509181014 and parameters: {'learning_rate': 0.04376255684556946, 'n_estimators': 58, 'colsample_bytree': 0.8059264473611898, 'subsample': 0.569746930326021, 'reg_alpha': 1.4607232426760908, 'reg_lambda': 1.8318092164684585}. Best is trial 1 with value: 173.9959086138952.
[I 2022-12-17 08:06:04,271] Trial 4 finished with value: 131.2024724349419 and parameters: {'learning_rate': 0.04615092843748656, 'n_estimators': 157, 'colsample_bytree': 0.5998368910791798, 'subsample': 0.7571172192068059, 'reg_alpha': 2.9620728443102124, 'reg_lambda': 0.23225206359998862}. Best is trial 1 with value: 173.9959086138952.
[I 2022-12-17 08:06:04,295] Trial 5 finished with value: 164.35616917844854 and parameters: {'learning_rate': 0.0611469403382424, 'n_estimators': 34, 'colsample_bytree': 0.5325257964926398, 'subsample': 0.9744427686266666, 'reg_alpha': 4.828160165372797, 'reg_lambda': 4.041986740582305}. Best is trial 1 with value: 173.9

Best trial : score 206.34682549731974, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:06:04,714] Trial 4 finished with value: 136.9623646423541 and parameters: {'learning_rate': 0.03178724984985056, 'n_estimators': 65, 'colsample_bytree': 0.864803089169032, 'subsample': 0.8187787356776066, 'reg_alpha': 4.436063712881633, 'reg_lambda': 2.3610746258097466}. Best is trial 2 with value: 244.8976394081565.
[I 2022-12-17 08:06:04,773] Trial 5 finished with value: 140.2457750469523 and parameters: {'learning_rate': 0.012839830347891868, 'n_estimators': 143, 'colsample_bytree': 0.8803925243084487, 'subsample': 0.7806385987847482, 'reg_alpha': 3.854835899772805, 'reg_lambda': 2.4689779818219537}. Best is trial 2 with value: 244.8976394081565.
[I 2022-12-17 08:06:04,814] Trial 6 finished with value: 129.73136466293653 and parameters: {'learning_rate': 0.052750550108817414, 'n_estimators': 85, 'colsample_bytree': 0.5127095633720475, 'subsample': 0.5539457134966522, 'reg_alpha': 0.15714592843367126, 'reg_lambda': 3.182052056318902}. Best is trial 2 with value: 244.

...

Best trial : score 256.4419243609132, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:06:05,701] Trial 3 finished with value: 166.01707083753226 and parameters: {'learning_rate': 0.08882155600224662, 'n_estimators': 156, 'colsample_bytree': 0.8210158230771438, 'subsample': 0.5420699824975244, 'reg_alpha': 0.8081435704730688, 'reg_lambda': 4.492770942635397}. Best is trial 0 with value: 187.45733827727477.
[I 2022-12-17 08:06:05,716] Trial 4 finished with value: 254.80050450593205 and parameters: {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}. Best is trial 4 with value: 254.80050450593205.
[I 2022-12-17 08:06:05,774] Trial 5 finished with value: 157.80608102538955 and parameters: {'learning_rate': 0.05532464514729203, 'n_estimators': 139, 'colsample_bytree': 0.8259806297513003, 'subsample': 0.61213465473028, 'reg_alpha': 3.560896106737679, 'reg_lambda': 1.1862454374840004}. Best is trial 4 with value: 2

Best trial : score 254.80050450593205, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:06:06,200] Trial 3 finished with value: 166.401036062875 and parameters: {'learning_rate': 0.05743605583884714, 'n_estimators': 19, 'colsample_bytree': 0.8075036133495849, 'subsample': 0.9950269250521316, 'reg_alpha': 0.7004200761826201, 'reg_lambda': 2.591648261818684}. Best is trial 3 with value: 166.401036062875.
[I 2022-12-17 08:06:06,260] Trial 4 finished with value: 142.46200115661856 and parameters: {'learning_rate': 0.0878599341208676, 'n_estimators': 148, 'colsample_bytree': 0.848507870497634, 'subsample': 0.8512420419935547, 'reg_alpha': 1.7974557560987758, 'reg_lambda': 1.4679592213224668}. Best is trial 3 with value: 166.401036062875.
[I 2022-12-17 08:06:06,326] Trial 5 finished with value: 142.73341654549145 and parameters: {'learning_rate': 0.08112675439237285, 'n_estimators': 162, 'colsample_bytree': 0.9335361592900518, 'subsample': 0.9566202762782356, 'reg_alpha': 2.556711994304689, 'reg_lambda': 2.507581473435998}. Best is trial 3 with value: 166.40103

Best trial : score 189.6534130865682, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [33]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Seodaemun_x, Seodaemun_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [34]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Seodaemun_x, Seodaemun_y)):
    n_iter += 1
    print(n_iter)
    train_x= Seodaemun_x.iloc[train_index]
    valid_x= Seodaemun_x.iloc[test_index]
    train_y= Seodaemun_y.iloc[train_index]
    valid_y= Seodaemun_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

#for model Seodaemun
#Best trial : score 239.70356623905482 
#params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}

0it [00:00, ?it/s][I 2022-12-17 08:06:14,483] A new study created in memory with name: no-name-c17240bc-7e01-4600-b28c-7abf6b838374
[I 2022-12-17 08:06:14,530] Trial 0 finished with value: 236.77124053824951 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 236.77124053824951.
[I 2022-12-17 08:06:14,569] Trial 1 finished with value: 240.85192601340765 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 240.85192601340765.
[I 2022-12-17 08:06:14,593] Trial 2 finished with value: 224.85570764326795 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.59091248360

1


[I 2022-12-17 08:06:14,656] Trial 4 finished with value: 233.5737088776835 and parameters: {'learning_rate': 0.04615092843748656, 'n_estimators': 157, 'colsample_bytree': 0.5998368910791798, 'subsample': 0.7571172192068059, 'reg_alpha': 2.9620728443102124, 'reg_lambda': 0.23225206359998862}. Best is trial 1 with value: 240.85192601340765.
[I 2022-12-17 08:06:14,674] Trial 5 finished with value: 222.87902669901118 and parameters: {'learning_rate': 0.0611469403382424, 'n_estimators': 34, 'colsample_bytree': 0.5325257964926398, 'subsample': 0.9744427686266666, 'reg_alpha': 4.828160165372797, 'reg_lambda': 4.041986740582305}. Best is trial 1 with value: 240.85192601340765.
[I 2022-12-17 08:06:14,692] Trial 6 finished with value: 245.72183016190525 and parameters: {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}. Best is trial 6 with value: 245.

Best trial : score 245.72183016190525, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:06:15,026] Trial 6 finished with value: 239.25034914277165 and parameters: {'learning_rate': 0.052750550108817414, 'n_estimators': 85, 'colsample_bytree': 0.5127095633720475, 'subsample': 0.5539457134966522, 'reg_alpha': 0.15714592843367126, 'reg_lambda': 3.182052056318902}. Best is trial 2 with value: 278.62725663386647.
[I 2022-12-17 08:06:15,056] Trial 7 finished with value: 232.86545169623673 and parameters: {'learning_rate': 0.03212124212655634, 'n_estimators': 102, 'colsample_bytree': 0.9537832369630466, 'subsample': 0.6246461145744375, 'reg_alpha': 2.0519146151781484, 'reg_lambda': 3.7777556927152434}. Best is trial 2 with value: 278.62725663386647.
[I 2022-12-17 08:06:15,073] Trial 8 finished with value: 275.99307751597183 and parameters: {'learning_rate': 0.023651018383670626, 'n_estimators': 15, 'colsample_bytree': 0.6448757264568841, 'subsample': 0.5806106436270022, 'reg_alpha': 4.648488261712865, 'reg_lambda': 4.040601897822085}. Best is trial 2 with value:

...

Best trial : score 298.23677525294744, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:06:15,670] Trial 6 finished with value: 155.32556476066446 and parameters: {'learning_rate': 0.03321457011776751, 'n_estimators': 150, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}. Best is trial 4 with value: 239.70356623905482.
[I 2022-12-17 08:06:15,696] Trial 7 finished with value: 194.90611581826784 and parameters: {'learning_rate': 0.010273802014981157, 'n_estimators': 73, 'colsample_bytree': 0.6326011838408627, 'subsample': 0.6219948216895418, 'reg_alpha': 4.865052773762228, 'reg_lambda': 1.965488623333802}. Best is trial 4 with value: 239.70356623905482.
[I 2022-12-17 08:06:15,740] Trial 8 finished with value: 157.94202159749528 and parameters: {'learning_rate': 0.08931260896253422, 'n_estimators': 126, 'colsample_bytree': 0.8974056517708242, 'subsample': 0.751318546552596, 'reg_alpha': 2.8845194231317954, 'reg_lambda': 2.4625884690943196}. Best is trial 4 with value: 23

Best trial : score 239.70356623905482, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:06:16,017] Trial 5 finished with value: 236.53516166908372 and parameters: {'learning_rate': 0.08112675439237285, 'n_estimators': 162, 'colsample_bytree': 0.9335361592900518, 'subsample': 0.9566202762782356, 'reg_alpha': 2.556711994304689, 'reg_lambda': 2.507581473435998}. Best is trial 3 with value: 279.93711402907246.
[I 2022-12-17 08:06:16,051] Trial 6 finished with value: 236.57202463577983 and parameters: {'learning_rate': 0.08003122271771075, 'n_estimators': 130, 'colsample_bytree': 0.8509834386288517, 'subsample': 0.8978963347180505, 'reg_alpha': 4.450026709087831, 'reg_lambda': 1.689975784257679}. Best is trial 3 with value: 279.93711402907246.
[I 2022-12-17 08:06:16,068] Trial 7 finished with value: 293.572263498942 and parameters: {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}. Best is trial 7 with value: 293.572

Best trial : score 293.572263498942, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [35]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Mapo_x, Mapo_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [36]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Mapo_x, Mapo_y)):
    n_iter += 1
    print(n_iter)
    train_x= Mapo_x.iloc[train_index]
    valid_x= Mapo_x.iloc[test_index]
    train_y= Mapo_y.iloc[train_index]
    valid_y= Mapo_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

#for model Mapo
#score 340.79072377178807, 
#params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}

0it [00:00, ?it/s][I 2022-12-17 08:06:27,228] A new study created in memory with name: no-name-e4de588a-1f5c-4214-81a7-7d802700ddce
[I 2022-12-17 08:06:27,298] Trial 0 finished with value: 246.23061214510372 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 246.23061214510372.
[I 2022-12-17 08:06:27,346] Trial 1 finished with value: 297.7083748137725 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 297.7083748137725.
[I 2022-12-17 08:06:27,367] Trial 2 finished with value: 247.78232560569816 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.5909124836035

1


[I 2022-12-17 08:06:27,398] Trial 3 finished with value: 256.1747577128743 and parameters: {'learning_rate': 0.04376255684556946, 'n_estimators': 58, 'colsample_bytree': 0.8059264473611898, 'subsample': 0.569746930326021, 'reg_alpha': 1.4607232426760908, 'reg_lambda': 1.8318092164684585}. Best is trial 1 with value: 297.7083748137725.
[I 2022-12-17 08:06:27,443] Trial 4 finished with value: 243.60192398076828 and parameters: {'learning_rate': 0.04615092843748656, 'n_estimators': 157, 'colsample_bytree': 0.5998368910791798, 'subsample': 0.7571172192068059, 'reg_alpha': 2.9620728443102124, 'reg_lambda': 0.23225206359998862}. Best is trial 1 with value: 297.7083748137725.
[I 2022-12-17 08:06:27,463] Trial 5 finished with value: 269.9525752838786 and parameters: {'learning_rate': 0.0611469403382424, 'n_estimators': 34, 'colsample_bytree': 0.5325257964926398, 'subsample': 0.9744427686266666, 'reg_alpha': 4.828160165372797, 'reg_lambda': 4.041986740582305}. Best is trial 1 with value: 297.70

Best trial : score 340.79072377178807, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:06:27,825] Trial 5 finished with value: 273.5371117905278 and parameters: {'learning_rate': 0.012839830347891868, 'n_estimators': 143, 'colsample_bytree': 0.8803925243084487, 'subsample': 0.7806385987847482, 'reg_alpha': 3.854835899772805, 'reg_lambda': 2.4689779818219537}. Best is trial 2 with value: 415.40537248371555.
[I 2022-12-17 08:06:27,856] Trial 6 finished with value: 235.6745867205963 and parameters: {'learning_rate': 0.052750550108817414, 'n_estimators': 85, 'colsample_bytree': 0.5127095633720475, 'subsample': 0.5539457134966522, 'reg_alpha': 0.15714592843367126, 'reg_lambda': 3.182052056318902}. Best is trial 2 with value: 415.40537248371555.
[I 2022-12-17 08:06:27,892] Trial 7 finished with value: 237.75258282731582 and parameters: {'learning_rate': 0.03212124212655634, 'n_estimators': 102, 'colsample_bytree': 0.9537832369630466, 'subsample': 0.6246461145744375, 'reg_alpha': 2.0519146151781484, 'reg_lambda': 3.7777556927152434}. Best is trial 2 with value:

...

Best trial : score 391.18838245692666, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:06:28,616] Trial 5 finished with value: 243.40441967407514 and parameters: {'learning_rate': 0.05532464514729203, 'n_estimators': 139, 'colsample_bytree': 0.8259806297513003, 'subsample': 0.61213465473028, 'reg_alpha': 3.560896106737679, 'reg_lambda': 1.1862454374840004}. Best is trial 4 with value: 394.2452100290968.
[I 2022-12-17 08:06:28,661] Trial 6 finished with value: 251.18195812296105 and parameters: {'learning_rate': 0.03321457011776751, 'n_estimators': 150, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}. Best is trial 4 with value: 394.2452100290968.
[I 2022-12-17 08:06:28,690] Trial 7 finished with value: 328.7668965184494 and parameters: {'learning_rate': 0.010273802014981157, 'n_estimators': 73, 'colsample_bytree': 0.6326011838408627, 'subsample': 0.6219948216895418, 'reg_alpha': 4.865052773762228, 'reg_lambda': 1.965488623333802}. Best is trial 4 with value: 394.245

Best trial : score 394.2452100290968, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:06:29,009] Trial 4 finished with value: 245.43729519204746 and parameters: {'learning_rate': 0.0878599341208676, 'n_estimators': 148, 'colsample_bytree': 0.848507870497634, 'subsample': 0.8512420419935547, 'reg_alpha': 1.7974557560987758, 'reg_lambda': 1.4679592213224668}. Best is trial 3 with value: 330.4360297666395.
[I 2022-12-17 08:06:29,056] Trial 5 finished with value: 248.2228697113519 and parameters: {'learning_rate': 0.08112675439237285, 'n_estimators': 162, 'colsample_bytree': 0.9335361592900518, 'subsample': 0.9566202762782356, 'reg_alpha': 2.556711994304689, 'reg_lambda': 2.507581473435998}. Best is trial 3 with value: 330.4360297666395.
[I 2022-12-17 08:06:29,097] Trial 6 finished with value: 243.04635419440723 and parameters: {'learning_rate': 0.08003122271771075, 'n_estimators': 130, 'colsample_bytree': 0.8509834386288517, 'subsample': 0.8978963347180505, 'reg_alpha': 4.450026709087831, 'reg_lambda': 1.689975784257679}. Best is trial 3 with value: 330.43

Best trial : score 370.15935281646483, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [37]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Yangcheon_x, Yangcheon_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [38]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Yangcheon_x, Yangcheon_y)):
    n_iter += 1
    print(n_iter)
    train_x= Yangcheon_x.iloc[train_index]
    valid_x= Yangcheon_x.iloc[test_index]
    train_y= Yangcheon_y.iloc[train_index]
    valid_y= Yangcheon_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

#for Yangcheon
#Best trial : score 311.17150847079114, 
#params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}

0it [00:00, ?it/s][I 2022-12-17 08:06:39,532] A new study created in memory with name: no-name-65fefad6-7b0f-4cd8-8231-0228a541e203
[I 2022-12-17 08:06:39,582] Trial 0 finished with value: 224.69700023547756 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 224.69700023547756.
[I 2022-12-17 08:06:39,630] Trial 1 finished with value: 269.5979304519672 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 269.5979304519672.
[I 2022-12-17 08:06:39,651] Trial 2 finished with value: 259.8627652764374 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.59091248360355

1


[I 2022-12-17 08:06:39,742] Trial 4 finished with value: 252.22858266658872 and parameters: {'learning_rate': 0.04615092843748656, 'n_estimators': 157, 'colsample_bytree': 0.5998368910791798, 'subsample': 0.7571172192068059, 'reg_alpha': 2.9620728443102124, 'reg_lambda': 0.23225206359998862}. Best is trial 1 with value: 269.5979304519672.
[I 2022-12-17 08:06:39,761] Trial 5 finished with value: 272.9023697097277 and parameters: {'learning_rate': 0.0611469403382424, 'n_estimators': 34, 'colsample_bytree': 0.5325257964926398, 'subsample': 0.9744427686266666, 'reg_alpha': 4.828160165372797, 'reg_lambda': 4.041986740582305}. Best is trial 5 with value: 272.9023697097277.
[I 2022-12-17 08:06:39,777] Trial 6 finished with value: 311.17150847079114 and parameters: {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}. Best is trial 6 with value: 311.17

Best trial : score 311.17150847079114, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:06:40,108] Trial 5 finished with value: 286.8759573861142 and parameters: {'learning_rate': 0.012839830347891868, 'n_estimators': 143, 'colsample_bytree': 0.8803925243084487, 'subsample': 0.7806385987847482, 'reg_alpha': 3.854835899772805, 'reg_lambda': 2.4689779818219537}. Best is trial 2 with value: 488.47325381074165.
[I 2022-12-17 08:06:40,144] Trial 6 finished with value: 264.66555901879997 and parameters: {'learning_rate': 0.052750550108817414, 'n_estimators': 85, 'colsample_bytree': 0.5127095633720475, 'subsample': 0.5539457134966522, 'reg_alpha': 0.15714592843367126, 'reg_lambda': 3.182052056318902}. Best is trial 2 with value: 488.47325381074165.
[I 2022-12-17 08:06:40,178] Trial 7 finished with value: 242.36546441837368 and parameters: {'learning_rate': 0.03212124212655634, 'n_estimators': 102, 'colsample_bytree': 0.9537832369630466, 'subsample': 0.6246461145744375, 'reg_alpha': 2.0519146151781484, 'reg_lambda': 3.7777556927152434}. Best is trial 2 with value

...

Best trial : score 459.3775696198335, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:06:40,877] Trial 5 finished with value: 275.49023036713345 and parameters: {'learning_rate': 0.05532464514729203, 'n_estimators': 139, 'colsample_bytree': 0.8259806297513003, 'subsample': 0.61213465473028, 'reg_alpha': 3.560896106737679, 'reg_lambda': 1.1862454374840004}. Best is trial 4 with value: 542.2410296247172.
[I 2022-12-17 08:06:40,922] Trial 6 finished with value: 267.8101238842017 and parameters: {'learning_rate': 0.03321457011776751, 'n_estimators': 150, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}. Best is trial 4 with value: 542.2410296247172.
[I 2022-12-17 08:06:40,949] Trial 7 finished with value: 409.8169328504277 and parameters: {'learning_rate': 0.010273802014981157, 'n_estimators': 73, 'colsample_bytree': 0.6326011838408627, 'subsample': 0.6219948216895418, 'reg_alpha': 4.865052773762228, 'reg_lambda': 1.965488623333802}. Best is trial 4 with value: 542.2410

Best trial : score 542.2410296247172, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:06:41,266] Trial 4 finished with value: 266.8139920659894 and parameters: {'learning_rate': 0.0878599341208676, 'n_estimators': 148, 'colsample_bytree': 0.848507870497634, 'subsample': 0.8512420419935547, 'reg_alpha': 1.7974557560987758, 'reg_lambda': 1.4679592213224668}. Best is trial 3 with value: 348.42462661311623.
[I 2022-12-17 08:06:41,327] Trial 5 finished with value: 267.62935856058886 and parameters: {'learning_rate': 0.08112675439237285, 'n_estimators': 162, 'colsample_bytree': 0.9335361592900518, 'subsample': 0.9566202762782356, 'reg_alpha': 2.556711994304689, 'reg_lambda': 2.507581473435998}. Best is trial 3 with value: 348.42462661311623.
[I 2022-12-17 08:06:41,366] Trial 6 finished with value: 266.40676085869825 and parameters: {'learning_rate': 0.08003122271771075, 'n_estimators': 130, 'colsample_bytree': 0.8509834386288517, 'subsample': 0.8978963347180505, 'reg_alpha': 4.450026709087831, 'reg_lambda': 1.689975784257679}. Best is trial 3 with value: 348.

Best trial : score 409.73115285160065, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [39]:
#Gangseo
#Guro
#Geumcheon
#Yeongdeungpo
#Dongjak
#Gwanak
#Seocho
#Gangnam
#Songpa
#Gangdong

In [40]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Gangseo_x, Gangseo_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [41]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Gangseo_x, Gangseo_y)):
    n_iter += 1
    print(n_iter)
    train_x= Gangseo_x.iloc[train_index]
    valid_x= Gangseo_x.iloc[test_index]
    train_y= Gangseo_y.iloc[train_index]
    valid_y= Gangseo_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))
#for Gangseo
#Best trial : score 223.26410978169906, 
#params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}

0it [00:00, ?it/s][I 2022-12-17 08:07:00,300] A new study created in memory with name: no-name-e7e46c4c-3920-4bd9-b200-2345aea10979
[I 2022-12-17 08:07:00,396] Trial 0 finished with value: 173.07982208941027 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 173.07982208941027.
[I 2022-12-17 08:07:00,458] Trial 1 finished with value: 200.32444186199967 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 200.32444186199967.


1


[I 2022-12-17 08:07:00,496] Trial 2 finished with value: 182.31579168315216 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.5909124836035503, 'subsample': 0.5917022549267169, 'reg_alpha': 1.5212112147976886, 'reg_lambda': 2.6237821581611893}. Best is trial 1 with value: 200.32444186199967.
[I 2022-12-17 08:07:00,528] Trial 3 finished with value: 177.7866436401884 and parameters: {'learning_rate': 0.04376255684556946, 'n_estimators': 58, 'colsample_bytree': 0.8059264473611898, 'subsample': 0.569746930326021, 'reg_alpha': 1.4607232426760908, 'reg_lambda': 1.8318092164684585}. Best is trial 1 with value: 200.32444186199967.
[I 2022-12-17 08:07:00,585] Trial 4 finished with value: 175.44956319068132 and parameters: {'learning_rate': 0.04615092843748656, 'n_estimators': 157, 'colsample_bytree': 0.5998368910791798, 'subsample': 0.7571172192068059, 'reg_alpha': 2.9620728443102124, 'reg_lambda': 0.23225206359998862}. Best is trial 1 with value: 

Best trial : score 223.22267811324593, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:07:01,036] Trial 4 finished with value: 207.86650379789313 and parameters: {'learning_rate': 0.03178724984985056, 'n_estimators': 65, 'colsample_bytree': 0.864803089169032, 'subsample': 0.8187787356776066, 'reg_alpha': 4.436063712881633, 'reg_lambda': 2.3610746258097466}. Best is trial 2 with value: 308.9578504649945.
[I 2022-12-17 08:07:01,092] Trial 5 finished with value: 214.45269055821225 and parameters: {'learning_rate': 0.012839830347891868, 'n_estimators': 143, 'colsample_bytree': 0.8803925243084487, 'subsample': 0.7806385987847482, 'reg_alpha': 3.854835899772805, 'reg_lambda': 2.4689779818219537}. Best is trial 2 with value: 308.9578504649945.
[I 2022-12-17 08:07:01,130] Trial 6 finished with value: 186.2611352419006 and parameters: {'learning_rate': 0.052750550108817414, 'n_estimators': 85, 'colsample_bytree': 0.5127095633720475, 'subsample': 0.5539457134966522, 'reg_alpha': 0.15714592843367126, 'reg_lambda': 3.182052056318902}. Best is trial 2 with value: 308

...

Best trial : score 290.4541821440467, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:07:02,028] Trial 4 finished with value: 311.2631171051576 and parameters: {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}. Best is trial 4 with value: 311.2631171051576.
[I 2022-12-17 08:07:02,099] Trial 5 finished with value: 209.34842629474286 and parameters: {'learning_rate': 0.05532464514729203, 'n_estimators': 139, 'colsample_bytree': 0.8259806297513003, 'subsample': 0.61213465473028, 'reg_alpha': 3.560896106737679, 'reg_lambda': 1.1862454374840004}. Best is trial 4 with value: 311.2631171051576.
[I 2022-12-17 08:07:02,159] Trial 6 finished with value: 209.92733299749284 and parameters: {'learning_rate': 0.03321457011776751, 'n_estimators': 150, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}. Best is trial 4 with value: 311.2

Best trial : score 311.2631171051576, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:07:02,548] Trial 2 finished with value: 211.8258288537119 and parameters: {'learning_rate': 0.08526253048016884, 'n_estimators': 63, 'colsample_bytree': 0.5847463733430462, 'subsample': 0.7784006312291751, 'reg_alpha': 4.680773870803905, 'reg_lambda': 3.480148983374865}. Best is trial 1 with value: 215.56319866730988.
[I 2022-12-17 08:07:02,570] Trial 3 finished with value: 244.50167291255272 and parameters: {'learning_rate': 0.05743605583884714, 'n_estimators': 19, 'colsample_bytree': 0.8075036133495849, 'subsample': 0.9950269250521316, 'reg_alpha': 0.7004200761826201, 'reg_lambda': 2.591648261818684}. Best is trial 3 with value: 244.50167291255272.
[I 2022-12-17 08:07:02,646] Trial 4 finished with value: 215.6858424176855 and parameters: {'learning_rate': 0.0878599341208676, 'n_estimators': 148, 'colsample_bytree': 0.848507870497634, 'subsample': 0.8512420419935547, 'reg_alpha': 1.7974557560987758, 'reg_lambda': 1.4679592213224668}. Best is trial 3 with value: 244.50

Best trial : score 260.74724072213564, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [42]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Guro_x, Guro_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [43]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Guro_x, Guro_y)):
    n_iter += 1
    print(n_iter)
    train_x= Guro_x.iloc[train_index]
    valid_x= Guro_x.iloc[test_index]
    train_y= Guro_y.iloc[train_index]
    valid_y= Guro_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))
#for Guro
#Best trial : score 186.12235007940922, 
#params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}

0it [00:00, ?it/s][I 2022-12-17 08:07:20,707] A new study created in memory with name: no-name-4145e9bb-be64-4533-b82e-4b7f20179614
[I 2022-12-17 08:07:20,778] Trial 0 finished with value: 158.09677289435524 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 158.09677289435524.
[I 2022-12-17 08:07:20,840] Trial 1 finished with value: 194.29949952599776 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 194.29949952599776.
[I 2022-12-17 08:07:20,865] Trial 2 finished with value: 173.18901682938352 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.59091248360

1


[I 2022-12-17 08:07:20,901] Trial 3 finished with value: 168.6831656867052 and parameters: {'learning_rate': 0.04376255684556946, 'n_estimators': 58, 'colsample_bytree': 0.8059264473611898, 'subsample': 0.569746930326021, 'reg_alpha': 1.4607232426760908, 'reg_lambda': 1.8318092164684585}. Best is trial 1 with value: 194.29949952599776.
[I 2022-12-17 08:07:20,963] Trial 4 finished with value: 168.6337371664723 and parameters: {'learning_rate': 0.04615092843748656, 'n_estimators': 157, 'colsample_bytree': 0.5998368910791798, 'subsample': 0.7571172192068059, 'reg_alpha': 2.9620728443102124, 'reg_lambda': 0.23225206359998862}. Best is trial 1 with value: 194.29949952599776.
[I 2022-12-17 08:07:20,988] Trial 5 finished with value: 186.3052094359189 and parameters: {'learning_rate': 0.0611469403382424, 'n_estimators': 34, 'colsample_bytree': 0.5325257964926398, 'subsample': 0.9744427686266666, 'reg_alpha': 4.828160165372797, 'reg_lambda': 4.041986740582305}. Best is trial 1 with value: 194.2

Best trial : score 211.88532028342735, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:07:21,429] Trial 4 finished with value: 168.30729056277227 and parameters: {'learning_rate': 0.03178724984985056, 'n_estimators': 65, 'colsample_bytree': 0.864803089169032, 'subsample': 0.8187787356776066, 'reg_alpha': 4.436063712881633, 'reg_lambda': 2.3610746258097466}. Best is trial 2 with value: 219.80694319862994.
[I 2022-12-17 08:07:21,485] Trial 5 finished with value: 169.77834664124978 and parameters: {'learning_rate': 0.012839830347891868, 'n_estimators': 143, 'colsample_bytree': 0.8803925243084487, 'subsample': 0.7806385987847482, 'reg_alpha': 3.854835899772805, 'reg_lambda': 2.4689779818219537}. Best is trial 2 with value: 219.80694319862994.
[I 2022-12-17 08:07:21,525] Trial 6 finished with value: 165.3643188587827 and parameters: {'learning_rate': 0.052750550108817414, 'n_estimators': 85, 'colsample_bytree': 0.5127095633720475, 'subsample': 0.5539457134966522, 'reg_alpha': 0.15714592843367126, 'reg_lambda': 3.182052056318902}. Best is trial 2 with value: 2

...

Best trial : score 260.12286117609824, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:07:22,411] Trial 4 finished with value: 236.6137716968099 and parameters: {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}. Best is trial 4 with value: 236.6137716968099.
[I 2022-12-17 08:07:22,475] Trial 5 finished with value: 161.6908096449808 and parameters: {'learning_rate': 0.05532464514729203, 'n_estimators': 139, 'colsample_bytree': 0.8259806297513003, 'subsample': 0.61213465473028, 'reg_alpha': 3.560896106737679, 'reg_lambda': 1.1862454374840004}. Best is trial 4 with value: 236.6137716968099.
[I 2022-12-17 08:07:22,533] Trial 6 finished with value: 159.924488945436 and parameters: {'learning_rate': 0.03321457011776751, 'n_estimators': 150, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}. Best is trial 4 with value: 236.6137

Best trial : score 236.6137716968099, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:07:22,916] Trial 3 finished with value: 166.57180109673996 and parameters: {'learning_rate': 0.05743605583884714, 'n_estimators': 19, 'colsample_bytree': 0.8075036133495849, 'subsample': 0.9950269250521316, 'reg_alpha': 0.7004200761826201, 'reg_lambda': 2.591648261818684}. Best is trial 3 with value: 166.57180109673996.
[I 2022-12-17 08:07:22,977] Trial 4 finished with value: 136.6342215693366 and parameters: {'learning_rate': 0.0878599341208676, 'n_estimators': 148, 'colsample_bytree': 0.848507870497634, 'subsample': 0.8512420419935547, 'reg_alpha': 1.7974557560987758, 'reg_lambda': 1.4679592213224668}. Best is trial 3 with value: 166.57180109673996.
[I 2022-12-17 08:07:23,042] Trial 5 finished with value: 133.35828469626597 and parameters: {'learning_rate': 0.08112675439237285, 'n_estimators': 162, 'colsample_bytree': 0.9335361592900518, 'subsample': 0.9566202762782356, 'reg_alpha': 2.556711994304689, 'reg_lambda': 2.507581473435998}. Best is trial 3 with value: 166.

Best trial : score 187.4649012574873, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [44]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Geumcheon_x, Geumcheon_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [45]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Geumcheon_x, Geumcheon_y)):
    n_iter += 1
    print(n_iter)
    train_x= Geumcheon_x.iloc[train_index]
    valid_x= Geumcheon_x.iloc[test_index]
    train_y= Geumcheon_y.iloc[train_index]
    valid_y= Geumcheon_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))
#for Geumcheon
#Best trial : score 159.3535254682672, 
#params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}

0it [00:00, ?it/s][I 2022-12-17 08:07:31,464] A new study created in memory with name: no-name-a19d376e-92e7-43b0-ae90-39df1db208f1
[I 2022-12-17 08:07:31,508] Trial 0 finished with value: 121.02301127214571 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 121.02301127214571.
[I 2022-12-17 08:07:31,547] Trial 1 finished with value: 154.50783235785707 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 154.50783235785707.
[I 2022-12-17 08:07:31,566] Trial 2 finished with value: 140.1768155915154 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.590912483603

1


[I 2022-12-17 08:07:31,646] Trial 5 finished with value: 147.48561921209063 and parameters: {'learning_rate': 0.0611469403382424, 'n_estimators': 34, 'colsample_bytree': 0.5325257964926398, 'subsample': 0.9744427686266666, 'reg_alpha': 4.828160165372797, 'reg_lambda': 4.041986740582305}. Best is trial 1 with value: 154.50783235785707.
[I 2022-12-17 08:07:31,663] Trial 6 finished with value: 159.2934914410661 and parameters: {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}. Best is trial 6 with value: 159.2934914410661.
[I 2022-12-17 08:07:31,704] Trial 7 finished with value: 156.46429385513235 and parameters: {'learning_rate': 0.004404463590406622, 'n_estimators': 182, 'colsample_bytree': 0.6293899908000085, 'subsample': 0.831261142176991, 'reg_alpha': 1.5585553804470549, 'reg_lambda': 2.600340105889054}. Best is trial 6 with value: 159.293

Best trial : score 159.2934914410661, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:07:31,986] Trial 6 finished with value: 146.90821760535292 and parameters: {'learning_rate': 0.052750550108817414, 'n_estimators': 85, 'colsample_bytree': 0.5127095633720475, 'subsample': 0.5539457134966522, 'reg_alpha': 0.15714592843367126, 'reg_lambda': 3.182052056318902}. Best is trial 2 with value: 196.37709407623004.
[I 2022-12-17 08:07:32,015] Trial 7 finished with value: 149.30328374767814 and parameters: {'learning_rate': 0.03212124212655634, 'n_estimators': 102, 'colsample_bytree': 0.9537832369630466, 'subsample': 0.6246461145744375, 'reg_alpha': 2.0519146151781484, 'reg_lambda': 3.7777556927152434}. Best is trial 2 with value: 196.37709407623004.
[I 2022-12-17 08:07:32,031] Trial 8 finished with value: 191.60511777136563 and parameters: {'learning_rate': 0.023651018383670626, 'n_estimators': 15, 'colsample_bytree': 0.6448757264568841, 'subsample': 0.5806106436270022, 'reg_alpha': 4.648488261712865, 'reg_lambda': 4.040601897822085}. Best is trial 2 with value:

...

Best trial : score 211.71628711021202, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:07:32,666] Trial 6 finished with value: 162.19426065474067 and parameters: {'learning_rate': 0.03321457011776751, 'n_estimators': 150, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}. Best is trial 4 with value: 211.6081403817236.
[I 2022-12-17 08:07:32,692] Trial 7 finished with value: 183.5884635951004 and parameters: {'learning_rate': 0.010273802014981157, 'n_estimators': 73, 'colsample_bytree': 0.6326011838408627, 'subsample': 0.6219948216895418, 'reg_alpha': 4.865052773762228, 'reg_lambda': 1.965488623333802}. Best is trial 4 with value: 211.6081403817236.
[I 2022-12-17 08:07:32,728] Trial 8 finished with value: 153.81041935299402 and parameters: {'learning_rate': 0.08931260896253422, 'n_estimators': 126, 'colsample_bytree': 0.8974056517708242, 'subsample': 0.751318546552596, 'reg_alpha': 2.8845194231317954, 'reg_lambda': 2.4625884690943196}. Best is trial 4 with value: 211.6

Best trial : score 211.6081403817236, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:07:33,001] Trial 5 finished with value: 150.4288393733994 and parameters: {'learning_rate': 0.08112675439237285, 'n_estimators': 162, 'colsample_bytree': 0.9335361592900518, 'subsample': 0.9566202762782356, 'reg_alpha': 2.556711994304689, 'reg_lambda': 2.507581473435998}. Best is trial 3 with value: 150.50057449533597.
[I 2022-12-17 08:07:33,038] Trial 6 finished with value: 148.40832558415914 and parameters: {'learning_rate': 0.08003122271771075, 'n_estimators': 130, 'colsample_bytree': 0.8509834386288517, 'subsample': 0.8978963347180505, 'reg_alpha': 4.450026709087831, 'reg_lambda': 1.689975784257679}. Best is trial 3 with value: 150.50057449533597.
[I 2022-12-17 08:07:33,054] Trial 7 finished with value: 160.62345031917306 and parameters: {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}. Best is trial 7 with value: 160.62

Best trial : score 160.62345031917306, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [46]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Yeongdeungpo_x, Yeongdeungpo_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [47]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Yeongdeungpo_x, Yeongdeungpo_y)):
    n_iter += 1
    print(n_iter)
    train_x= Yeongdeungpo_x.iloc[train_index]
    valid_x= Yeongdeungpo_x.iloc[test_index]
    train_y= Yeongdeungpo_y.iloc[train_index]
    valid_y= Yeongdeungpo_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))
#for Yeongdeungpo
#Best trial : score 282.1275828372063, 
#params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}

0it [00:00, ?it/s][I 2022-12-17 08:07:47,313] A new study created in memory with name: no-name-bc1b88ad-5bf1-4642-acf2-911cd8bf0fe0
[I 2022-12-17 08:07:47,368] Trial 0 finished with value: 220.1048331885129 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 220.1048331885129.
[I 2022-12-17 08:07:47,416] Trial 1 finished with value: 290.3603244733661 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 290.3603244733661.
[I 2022-12-17 08:07:47,437] Trial 2 finished with value: 217.23729992782586 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.590912483603550

1


[I 2022-12-17 08:07:47,513] Trial 4 finished with value: 212.77679767112036 and parameters: {'learning_rate': 0.04615092843748656, 'n_estimators': 157, 'colsample_bytree': 0.5998368910791798, 'subsample': 0.7571172192068059, 'reg_alpha': 2.9620728443102124, 'reg_lambda': 0.23225206359998862}. Best is trial 1 with value: 290.3603244733661.
[I 2022-12-17 08:07:47,532] Trial 5 finished with value: 238.7276503404068 and parameters: {'learning_rate': 0.0611469403382424, 'n_estimators': 34, 'colsample_bytree': 0.5325257964926398, 'subsample': 0.9744427686266666, 'reg_alpha': 4.828160165372797, 'reg_lambda': 4.041986740582305}. Best is trial 1 with value: 290.3603244733661.
[I 2022-12-17 08:07:47,549] Trial 6 finished with value: 347.0645091809866 and parameters: {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}. Best is trial 6 with value: 347.064

Best trial : score 347.0645091809866, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:07:47,905] Trial 5 finished with value: 246.35416649224084 and parameters: {'learning_rate': 0.012839830347891868, 'n_estimators': 143, 'colsample_bytree': 0.8803925243084487, 'subsample': 0.7806385987847482, 'reg_alpha': 3.854835899772805, 'reg_lambda': 2.4689779818219537}. Best is trial 2 with value: 440.31597423372426.
[I 2022-12-17 08:07:47,935] Trial 6 finished with value: 199.68337917858668 and parameters: {'learning_rate': 0.052750550108817414, 'n_estimators': 85, 'colsample_bytree': 0.5127095633720475, 'subsample': 0.5539457134966522, 'reg_alpha': 0.15714592843367126, 'reg_lambda': 3.182052056318902}. Best is trial 2 with value: 440.31597423372426.
[I 2022-12-17 08:07:47,981] Trial 7 finished with value: 207.18638608221931 and parameters: {'learning_rate': 0.03212124212655634, 'n_estimators': 102, 'colsample_bytree': 0.9537832369630466, 'subsample': 0.6246461145744375, 'reg_alpha': 2.0519146151781484, 'reg_lambda': 3.7777556927152434}. Best is trial 2 with valu

...

Best trial : score 383.95212914980567, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:07:48,724] Trial 5 finished with value: 171.32004000222014 and parameters: {'learning_rate': 0.05532464514729203, 'n_estimators': 139, 'colsample_bytree': 0.8259806297513003, 'subsample': 0.61213465473028, 'reg_alpha': 3.560896106737679, 'reg_lambda': 1.1862454374840004}. Best is trial 4 with value: 437.5639364583316.
[I 2022-12-17 08:07:48,769] Trial 6 finished with value: 168.0104255690479 and parameters: {'learning_rate': 0.03321457011776751, 'n_estimators': 150, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}. Best is trial 4 with value: 437.5639364583316.
[I 2022-12-17 08:07:48,796] Trial 7 finished with value: 288.21946196582593 and parameters: {'learning_rate': 0.010273802014981157, 'n_estimators': 73, 'colsample_bytree': 0.6326011838408627, 'subsample': 0.6219948216895418, 'reg_alpha': 4.865052773762228, 'reg_lambda': 1.965488623333802}. Best is trial 4 with value: 437.563

Best trial : score 437.5639364583316, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:07:49,133] Trial 4 finished with value: 189.6454532927191 and parameters: {'learning_rate': 0.0878599341208676, 'n_estimators': 148, 'colsample_bytree': 0.848507870497634, 'subsample': 0.8512420419935547, 'reg_alpha': 1.7974557560987758, 'reg_lambda': 1.4679592213224668}. Best is trial 3 with value: 247.47153299867236.
[I 2022-12-17 08:07:49,184] Trial 5 finished with value: 186.65933951560888 and parameters: {'learning_rate': 0.08112675439237285, 'n_estimators': 162, 'colsample_bytree': 0.9335361592900518, 'subsample': 0.9566202762782356, 'reg_alpha': 2.556711994304689, 'reg_lambda': 2.507581473435998}. Best is trial 3 with value: 247.47153299867236.
[I 2022-12-17 08:07:49,228] Trial 6 finished with value: 191.15487395712952 and parameters: {'learning_rate': 0.08003122271771075, 'n_estimators': 130, 'colsample_bytree': 0.8509834386288517, 'subsample': 0.8978963347180505, 'reg_alpha': 4.450026709087831, 'reg_lambda': 1.689975784257679}. Best is trial 3 with value: 247.

Best trial : score 283.48590097597486, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [48]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Dongjak_x, Dongjak_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [49]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Dongjak_x, Dongjak_y)):
    n_iter += 1
    print(n_iter)
    train_x= Dongjak_x.iloc[train_index]
    valid_x= Dongjak_x.iloc[test_index]
    train_y= Dongjak_y.iloc[train_index]
    valid_y= Dongjak_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))
#for Dongjak
#Best trial : score 242.54260711039302, 
#params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}

0it [00:00, ?it/s][I 2022-12-17 08:08:04,147] A new study created in memory with name: no-name-360247d3-28bb-4297-a3a5-933d6467e0ec
[I 2022-12-17 08:08:04,213] Trial 0 finished with value: 214.99277417221097 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 214.99277417221097.
[I 2022-12-17 08:08:04,269] Trial 1 finished with value: 261.08372436091935 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 261.08372436091935.
[I 2022-12-17 08:08:04,292] Trial 2 finished with value: 230.01817919968659 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.59091248360

1


[I 2022-12-17 08:08:04,323] Trial 3 finished with value: 233.23965742951808 and parameters: {'learning_rate': 0.04376255684556946, 'n_estimators': 58, 'colsample_bytree': 0.8059264473611898, 'subsample': 0.569746930326021, 'reg_alpha': 1.4607232426760908, 'reg_lambda': 1.8318092164684585}. Best is trial 1 with value: 261.08372436091935.
[I 2022-12-17 08:08:04,371] Trial 4 finished with value: 217.84300415911147 and parameters: {'learning_rate': 0.04615092843748656, 'n_estimators': 157, 'colsample_bytree': 0.5998368910791798, 'subsample': 0.7571172192068059, 'reg_alpha': 2.9620728443102124, 'reg_lambda': 0.23225206359998862}. Best is trial 1 with value: 261.08372436091935.
[I 2022-12-17 08:08:04,391] Trial 5 finished with value: 243.49185630157032 and parameters: {'learning_rate': 0.0611469403382424, 'n_estimators': 34, 'colsample_bytree': 0.5325257964926398, 'subsample': 0.9744427686266666, 'reg_alpha': 4.828160165372797, 'reg_lambda': 4.041986740582305}. Best is trial 1 with value: 26

Best trial : score 286.1114748170952, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:08:04,762] Trial 4 finished with value: 194.05483844399626 and parameters: {'learning_rate': 0.03178724984985056, 'n_estimators': 65, 'colsample_bytree': 0.864803089169032, 'subsample': 0.8187787356776066, 'reg_alpha': 4.436063712881633, 'reg_lambda': 2.3610746258097466}. Best is trial 2 with value: 268.7514409038405.
[I 2022-12-17 08:08:04,814] Trial 5 finished with value: 195.97952102761872 and parameters: {'learning_rate': 0.012839830347891868, 'n_estimators': 143, 'colsample_bytree': 0.8803925243084487, 'subsample': 0.7806385987847482, 'reg_alpha': 3.854835899772805, 'reg_lambda': 2.4689779818219537}. Best is trial 2 with value: 268.7514409038405.
[I 2022-12-17 08:08:04,857] Trial 6 finished with value: 180.26513619589355 and parameters: {'learning_rate': 0.052750550108817414, 'n_estimators': 85, 'colsample_bytree': 0.5127095633720475, 'subsample': 0.5539457134966522, 'reg_alpha': 0.15714592843367126, 'reg_lambda': 3.182052056318902}. Best is trial 2 with value: 26

...

Best trial : score 282.5196718031128, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:08:05,668] Trial 5 finished with value: 153.93464635096956 and parameters: {'learning_rate': 0.05532464514729203, 'n_estimators': 139, 'colsample_bytree': 0.8259806297513003, 'subsample': 0.61213465473028, 'reg_alpha': 3.560896106737679, 'reg_lambda': 1.1862454374840004}. Best is trial 4 with value: 254.15507768036014.
[I 2022-12-17 08:08:05,720] Trial 6 finished with value: 166.90719380641843 and parameters: {'learning_rate': 0.03321457011776751, 'n_estimators': 150, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}. Best is trial 4 with value: 254.15507768036014.
[I 2022-12-17 08:08:05,750] Trial 7 finished with value: 210.65949822866475 and parameters: {'learning_rate': 0.010273802014981157, 'n_estimators': 73, 'colsample_bytree': 0.6326011838408627, 'subsample': 0.6219948216895418, 'reg_alpha': 4.865052773762228, 'reg_lambda': 1.965488623333802}. Best is trial 4 with value: 254.

Best trial : score 254.15507768036014, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:08:06,097] Trial 4 finished with value: 212.28515792291392 and parameters: {'learning_rate': 0.0878599341208676, 'n_estimators': 148, 'colsample_bytree': 0.848507870497634, 'subsample': 0.8512420419935547, 'reg_alpha': 1.7974557560987758, 'reg_lambda': 1.4679592213224668}. Best is trial 3 with value: 229.18598497427945.
[I 2022-12-17 08:08:06,154] Trial 5 finished with value: 210.05146382284934 and parameters: {'learning_rate': 0.08112675439237285, 'n_estimators': 162, 'colsample_bytree': 0.9335361592900518, 'subsample': 0.9566202762782356, 'reg_alpha': 2.556711994304689, 'reg_lambda': 2.507581473435998}. Best is trial 3 with value: 229.18598497427945.
[I 2022-12-17 08:08:06,202] Trial 6 finished with value: 212.35232484991366 and parameters: {'learning_rate': 0.08003122271771075, 'n_estimators': 130, 'colsample_bytree': 0.8509834386288517, 'subsample': 0.8978963347180505, 'reg_alpha': 4.450026709087831, 'reg_lambda': 1.689975784257679}. Best is trial 3 with value: 229

Best trial : score 242.86351757610342, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [50]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Gwanak_x, Gwanak_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [51]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Gwanak_x, Gwanak_y)):
    n_iter += 1
    print(n_iter)
    train_x= Gwanak_x.iloc[train_index]
    valid_x= Gwanak_x.iloc[test_index]
    train_y= Gwanak_y.iloc[train_index]
    valid_y= Gwanak_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))
#for Gwanak
#Best trial : score 161.62117226207127, 
#params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}

0it [00:00, ?it/s][I 2022-12-17 08:08:18,395] A new study created in memory with name: no-name-bf0d15d1-f888-4e3c-9795-9e9b9e595dfe
[I 2022-12-17 08:08:18,450] Trial 0 finished with value: 147.75759550225126 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 147.75759550225126.
[I 2022-12-17 08:08:18,502] Trial 1 finished with value: 170.72604728997626 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 170.72604728997626.
[I 2022-12-17 08:08:18,523] Trial 2 finished with value: 160.99724453358152 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.59091248360

1


[I 2022-12-17 08:08:18,597] Trial 4 finished with value: 155.45265842212126 and parameters: {'learning_rate': 0.04615092843748656, 'n_estimators': 157, 'colsample_bytree': 0.5998368910791798, 'subsample': 0.7571172192068059, 'reg_alpha': 2.9620728443102124, 'reg_lambda': 0.23225206359998862}. Best is trial 1 with value: 170.72604728997626.
[I 2022-12-17 08:08:18,616] Trial 5 finished with value: 166.0918711041095 and parameters: {'learning_rate': 0.0611469403382424, 'n_estimators': 34, 'colsample_bytree': 0.5325257964926398, 'subsample': 0.9744427686266666, 'reg_alpha': 4.828160165372797, 'reg_lambda': 4.041986740582305}. Best is trial 1 with value: 170.72604728997626.
[I 2022-12-17 08:08:18,633] Trial 6 finished with value: 178.56880026942414 and parameters: {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}. Best is trial 6 with value: 178.

Best trial : score 178.56880026942414, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:08:18,991] Trial 5 finished with value: 167.65230069239303 and parameters: {'learning_rate': 0.012839830347891868, 'n_estimators': 143, 'colsample_bytree': 0.8803925243084487, 'subsample': 0.7806385987847482, 'reg_alpha': 3.854835899772805, 'reg_lambda': 2.4689779818219537}. Best is trial 2 with value: 175.5286287414059.
[I 2022-12-17 08:08:19,023] Trial 6 finished with value: 166.03852080573185 and parameters: {'learning_rate': 0.052750550108817414, 'n_estimators': 85, 'colsample_bytree': 0.5127095633720475, 'subsample': 0.5539457134966522, 'reg_alpha': 0.15714592843367126, 'reg_lambda': 3.182052056318902}. Best is trial 2 with value: 175.5286287414059.
[I 2022-12-17 08:08:19,059] Trial 7 finished with value: 171.5773240338898 and parameters: {'learning_rate': 0.03212124212655634, 'n_estimators': 102, 'colsample_bytree': 0.9537832369630466, 'subsample': 0.6246461145744375, 'reg_alpha': 2.0519146151781484, 'reg_lambda': 3.7777556927152434}. Best is trial 2 with value: 

...

Best trial : score 177.43978098288773, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:08:19,782] Trial 5 finished with value: 166.0026728432722 and parameters: {'learning_rate': 0.05532464514729203, 'n_estimators': 139, 'colsample_bytree': 0.8259806297513003, 'subsample': 0.61213465473028, 'reg_alpha': 3.560896106737679, 'reg_lambda': 1.1862454374840004}. Best is trial 4 with value: 205.00273515686493.
[I 2022-12-17 08:08:19,830] Trial 6 finished with value: 168.9137398813624 and parameters: {'learning_rate': 0.03321457011776751, 'n_estimators': 150, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}. Best is trial 4 with value: 205.00273515686493.
[I 2022-12-17 08:08:19,858] Trial 7 finished with value: 190.5544838730644 and parameters: {'learning_rate': 0.010273802014981157, 'n_estimators': 73, 'colsample_bytree': 0.6326011838408627, 'subsample': 0.6219948216895418, 'reg_alpha': 4.865052773762228, 'reg_lambda': 1.965488623333802}. Best is trial 4 with value: 205.002

Best trial : score 205.00273515686493, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:08:20,186] Trial 4 finished with value: 153.49345966828974 and parameters: {'learning_rate': 0.0878599341208676, 'n_estimators': 148, 'colsample_bytree': 0.848507870497634, 'subsample': 0.8512420419935547, 'reg_alpha': 1.7974557560987758, 'reg_lambda': 1.4679592213224668}. Best is trial 3 with value: 155.37891102394155.
[I 2022-12-17 08:08:20,240] Trial 5 finished with value: 150.08960911090168 and parameters: {'learning_rate': 0.08112675439237285, 'n_estimators': 162, 'colsample_bytree': 0.9335361592900518, 'subsample': 0.9566202762782356, 'reg_alpha': 2.556711994304689, 'reg_lambda': 2.507581473435998}. Best is trial 3 with value: 155.37891102394155.
[I 2022-12-17 08:08:20,282] Trial 6 finished with value: 146.742324695607 and parameters: {'learning_rate': 0.08003122271771075, 'n_estimators': 130, 'colsample_bytree': 0.8509834386288517, 'subsample': 0.8978963347180505, 'reg_alpha': 4.450026709087831, 'reg_lambda': 1.689975784257679}. Best is trial 3 with value: 155.3

Best trial : score 162.00067365706806, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [52]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Seocho_x, Seocho_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [53]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Seocho_x, Seocho_y)):
    n_iter += 1
    print(n_iter)
    train_x= Seocho_x.iloc[train_index]
    valid_x= Seocho_x.iloc[test_index]
    train_y= Seocho_y.iloc[train_index]
    valid_y= Seocho_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))
#for Seocho
#Best trial : score 676.45612671652, 
#params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}

0it [00:00, ?it/s][I 2022-12-17 08:08:39,994] A new study created in memory with name: no-name-bf216e2d-5f47-4a3f-ba1b-e1d3964e1025
[I 2022-12-17 08:08:40,079] Trial 0 finished with value: 335.9533735962739 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 335.9533735962739.
[I 2022-12-17 08:08:40,133] Trial 1 finished with value: 494.0081361648596 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 494.0081361648596.


1


[I 2022-12-17 08:08:40,158] Trial 2 finished with value: 367.75895479805354 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.5909124836035503, 'subsample': 0.5917022549267169, 'reg_alpha': 1.5212112147976886, 'reg_lambda': 2.6237821581611893}. Best is trial 1 with value: 494.0081361648596.
[I 2022-12-17 08:08:40,191] Trial 3 finished with value: 359.83540000573123 and parameters: {'learning_rate': 0.04376255684556946, 'n_estimators': 58, 'colsample_bytree': 0.8059264473611898, 'subsample': 0.569746930326021, 'reg_alpha': 1.4607232426760908, 'reg_lambda': 1.8318092164684585}. Best is trial 1 with value: 494.0081361648596.
[I 2022-12-17 08:08:40,249] Trial 4 finished with value: 349.0377164848163 and parameters: {'learning_rate': 0.04615092843748656, 'n_estimators': 157, 'colsample_bytree': 0.5998368910791798, 'subsample': 0.7571172192068059, 'reg_alpha': 2.9620728443102124, 'reg_lambda': 0.23225206359998862}. Best is trial 1 with value: 49

Best trial : score 676.8186807567903, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:08:40,662] Trial 4 finished with value: 454.66012389617725 and parameters: {'learning_rate': 0.03178724984985056, 'n_estimators': 65, 'colsample_bytree': 0.864803089169032, 'subsample': 0.8187787356776066, 'reg_alpha': 4.436063712881633, 'reg_lambda': 2.3610746258097466}. Best is trial 2 with value: 890.2874391237691.
[I 2022-12-17 08:08:40,725] Trial 5 finished with value: 475.19785148493827 and parameters: {'learning_rate': 0.012839830347891868, 'n_estimators': 143, 'colsample_bytree': 0.8803925243084487, 'subsample': 0.7806385987847482, 'reg_alpha': 3.854835899772805, 'reg_lambda': 2.4689779818219537}. Best is trial 2 with value: 890.2874391237691.
[I 2022-12-17 08:08:40,775] Trial 6 finished with value: 398.89797400325176 and parameters: {'learning_rate': 0.052750550108817414, 'n_estimators': 85, 'colsample_bytree': 0.5127095633720475, 'subsample': 0.5539457134966522, 'reg_alpha': 0.15714592843367126, 'reg_lambda': 3.182052056318902}. Best is trial 2 with value: 89

...

Best trial : score 951.6508915827511, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:08:41,683] Trial 4 finished with value: 1078.7272496368514 and parameters: {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}. Best is trial 4 with value: 1078.7272496368514.
[I 2022-12-17 08:08:41,752] Trial 5 finished with value: 373.41141792111034 and parameters: {'learning_rate': 0.05532464514729203, 'n_estimators': 139, 'colsample_bytree': 0.8259806297513003, 'subsample': 0.61213465473028, 'reg_alpha': 3.560896106737679, 'reg_lambda': 1.1862454374840004}. Best is trial 4 with value: 1078.7272496368514.
[I 2022-12-17 08:08:41,828] Trial 6 finished with value: 373.28473700188385 and parameters: {'learning_rate': 0.03321457011776751, 'n_estimators': 150, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}. Best is trial 4 with value: 10

Best trial : score 1078.7272496368514, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:08:42,189] Trial 2 finished with value: 484.90370712047155 and parameters: {'learning_rate': 0.08526253048016884, 'n_estimators': 63, 'colsample_bytree': 0.5847463733430462, 'subsample': 0.7784006312291751, 'reg_alpha': 4.680773870803905, 'reg_lambda': 3.480148983374865}. Best is trial 2 with value: 484.90370712047155.
[I 2022-12-17 08:08:42,209] Trial 3 finished with value: 613.8396933582251 and parameters: {'learning_rate': 0.05743605583884714, 'n_estimators': 19, 'colsample_bytree': 0.8075036133495849, 'subsample': 0.9950269250521316, 'reg_alpha': 0.7004200761826201, 'reg_lambda': 2.591648261818684}. Best is trial 3 with value: 613.8396933582251.
[I 2022-12-17 08:08:42,272] Trial 4 finished with value: 426.6394126687914 and parameters: {'learning_rate': 0.0878599341208676, 'n_estimators': 148, 'colsample_bytree': 0.848507870497634, 'subsample': 0.8512420419935547, 'reg_alpha': 1.7974557560987758, 'reg_lambda': 1.4679592213224668}. Best is trial 3 with value: 613.839

Best trial : score 726.5625415466029, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [54]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Gangnam_x, Gangnam_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [55]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Gangnam_x, Gangnam_y)):
    n_iter += 1
    print(n_iter)
    train_x= Gangnam_x.iloc[train_index]
    valid_x= Gangnam_x.iloc[test_index]
    train_y= Gangnam_y.iloc[train_index]
    valid_y= Gangnam_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))
#for Gangnam
#Best trial : score 635.4383335938923, 
#params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}

0it [00:00, ?it/s][I 2022-12-17 08:09:07,370] A new study created in memory with name: no-name-77a33a31-5eff-4e32-ad8f-dcf1a3cafde5
[I 2022-12-17 08:09:07,455] Trial 0 finished with value: 419.60532988149015 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 419.60532988149015.
[I 2022-12-17 08:09:07,528] Trial 1 finished with value: 591.9846876743159 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 591.9846876743159.


1


[I 2022-12-17 08:09:07,561] Trial 2 finished with value: 489.27493976635895 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.5909124836035503, 'subsample': 0.5917022549267169, 'reg_alpha': 1.5212112147976886, 'reg_lambda': 2.6237821581611893}. Best is trial 1 with value: 591.9846876743159.
[I 2022-12-17 08:09:07,712] Trial 3 finished with value: 461.01531411041043 and parameters: {'learning_rate': 0.04376255684556946, 'n_estimators': 58, 'colsample_bytree': 0.8059264473611898, 'subsample': 0.569746930326021, 'reg_alpha': 1.4607232426760908, 'reg_lambda': 1.8318092164684585}. Best is trial 1 with value: 591.9846876743159.
[I 2022-12-17 08:09:07,780] Trial 4 finished with value: 438.6696965760697 and parameters: {'learning_rate': 0.04615092843748656, 'n_estimators': 157, 'colsample_bytree': 0.5998368910791798, 'subsample': 0.7571172192068059, 'reg_alpha': 2.9620728443102124, 'reg_lambda': 0.23225206359998862}. Best is trial 1 with value: 59

Best trial : score 696.3364979563454, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:09:08,235] Trial 3 finished with value: 613.7334775401844 and parameters: {'learning_rate': 0.03648810712588299, 'n_estimators': 23, 'colsample_bytree': 0.9315517129377968, 'subsample': 0.811649063413779, 'reg_alpha': 1.654490124263246, 'reg_lambda': 0.3177917514301182}. Best is trial 2 with value: 773.2031363811342.
[I 2022-12-17 08:09:08,273] Trial 4 finished with value: 514.0745591224652 and parameters: {'learning_rate': 0.03178724984985056, 'n_estimators': 65, 'colsample_bytree': 0.864803089169032, 'subsample': 0.8187787356776066, 'reg_alpha': 4.436063712881633, 'reg_lambda': 2.3610746258097466}. Best is trial 2 with value: 773.2031363811342.
[I 2022-12-17 08:09:08,340] Trial 5 finished with value: 531.5811986871537 and parameters: {'learning_rate': 0.012839830347891868, 'n_estimators': 143, 'colsample_bytree': 0.8803925243084487, 'subsample': 0.7806385987847482, 'reg_alpha': 3.854835899772805, 'reg_lambda': 2.4689779818219537}. Best is trial 2 with value: 773.2031

...

Best trial : score 738.8344666874672, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:09:09,416] Trial 3 finished with value: 443.1544026208308 and parameters: {'learning_rate': 0.08882155600224662, 'n_estimators': 156, 'colsample_bytree': 0.8210158230771438, 'subsample': 0.5420699824975244, 'reg_alpha': 0.8081435704730688, 'reg_lambda': 4.492770942635397}. Best is trial 0 with value: 565.3226793124006.
[I 2022-12-17 08:09:09,430] Trial 4 finished with value: 743.061722353759 and parameters: {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}. Best is trial 4 with value: 743.061722353759.
[I 2022-12-17 08:09:09,496] Trial 5 finished with value: 436.08276750992746 and parameters: {'learning_rate': 0.05532464514729203, 'n_estimators': 139, 'colsample_bytree': 0.8259806297513003, 'subsample': 0.61213465473028, 'reg_alpha': 3.560896106737679, 'reg_lambda': 1.1862454374840004}. Best is trial 4 with value: 743.061

Best trial : score 743.061722353759, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:09:09,980] Trial 1 finished with value: 443.55338453666883 and parameters: {'learning_rate': 0.04339023068341412, 'n_estimators': 194, 'colsample_bytree': 0.9818099885446264, 'subsample': 0.92650472773368, 'reg_alpha': 1.4722444603479286, 'reg_lambda': 1.9254886430096263}. Best is trial 1 with value: 443.55338453666883.
[I 2022-12-17 08:09:10,017] Trial 2 finished with value: 470.60155209520497 and parameters: {'learning_rate': 0.08526253048016884, 'n_estimators': 63, 'colsample_bytree': 0.5847463733430462, 'subsample': 0.7784006312291751, 'reg_alpha': 4.680773870803905, 'reg_lambda': 3.480148983374865}. Best is trial 2 with value: 470.60155209520497.
[I 2022-12-17 08:09:10,039] Trial 3 finished with value: 561.8967392411687 and parameters: {'learning_rate': 0.05743605583884714, 'n_estimators': 19, 'colsample_bytree': 0.8075036133495849, 'subsample': 0.9950269250521316, 'reg_alpha': 0.7004200761826201, 'reg_lambda': 2.591648261818684}. Best is trial 3 with value: 561.8

Best trial : score 637.4338405631867, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}


In [56]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Songpa_x, Songpa_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 200}
LGBMRegressor(n_estimators=200)


In [57]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Songpa_x, Songpa_y)):
    n_iter += 1
    print(n_iter)
    train_x= Songpa_x.iloc[train_index]
    valid_x= Songpa_x.iloc[test_index]
    train_y= Songpa_y.iloc[train_index]
    valid_y= Songpa_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))
#for Songpa
#Best trial : score 284.5757976709995, 
#params {'learning_rate': 0.0878599341208676, 'n_estimators': 248, 'colsample_bytree': 0.848507870497634, 'subsample': 0.8512420419935547, 'reg_alpha': 1.7974557560987758, 'reg_lambda': 1.4679592213224668}

0it [00:00, ?it/s][I 2022-12-17 08:09:27,066] A new study created in memory with name: no-name-bed23b64-c38e-46e7-a4df-2786cca1471b
[I 2022-12-17 08:09:27,170] Trial 0 finished with value: 263.8977745157306 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 291, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 263.8977745157306.


1


[I 2022-12-17 08:09:27,257] Trial 1 finished with value: 338.34775138040527 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 274, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 338.34775138040527.
[I 2022-12-17 08:09:27,314] Trial 2 finished with value: 256.30786641966483 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 142, 'colsample_bytree': 0.5909124836035503, 'subsample': 0.5917022549267169, 'reg_alpha': 1.5212112147976886, 'reg_lambda': 2.6237821581611893}. Best is trial 1 with value: 338.34775138040527.
[I 2022-12-17 08:09:27,390] Trial 3 finished with value: 271.2926928647442 and parameters: {'learning_rate': 0.04376255684556946, 'n_estimators': 158, 'colsample_bytree': 0.8059264473611898, 'subsample': 0.569746930326021, 'reg_alpha': 1.4607232426760908, 'reg_lambda': 1.8318092164684585}. Best is trial 1 with valu

Best trial : score 381.1136293194275, 
params {'learning_rate': 0.004404463590406622, 'n_estimators': 282, 'colsample_bytree': 0.6293899908000085, 'subsample': 0.831261142176991, 'reg_alpha': 1.5585553804470549, 'reg_lambda': 2.600340105889054}
2


[I 2022-12-17 08:09:28,076] Trial 2 finished with value: 508.69642942112034 and parameters: {'learning_rate': 0.0015466895952366377, 'n_estimators': 263, 'colsample_bytree': 0.8534286719238086, 'subsample': 0.8645035840204937, 'reg_alpha': 3.8563517334297286, 'reg_lambda': 0.3702232586704518}. Best is trial 2 with value: 508.69642942112034.
[I 2022-12-17 08:09:28,137] Trial 3 finished with value: 271.50004035263146 and parameters: {'learning_rate': 0.03648810712588299, 'n_estimators': 123, 'colsample_bytree': 0.9315517129377968, 'subsample': 0.811649063413779, 'reg_alpha': 1.654490124263246, 'reg_lambda': 0.3177917514301182}. Best is trial 2 with value: 508.69642942112034.
[I 2022-12-17 08:09:28,197] Trial 4 finished with value: 269.938417396565 and parameters: {'learning_rate': 0.03178724984985056, 'n_estimators': 165, 'colsample_bytree': 0.864803089169032, 'subsample': 0.8187787356776066, 'reg_alpha': 4.436063712881633, 'reg_lambda': 2.3610746258097466}. Best is trial 2 with value: 5

...

Best trial : score 377.2280462637351, 
params {'learning_rate': 0.005036739013921628, 'n_estimators': 218, 'colsample_bytree': 0.8387821809211412, 'subsample': 0.508293914463928, 'reg_alpha': 2.560465291496405, 'reg_lambda': 1.1324788759896898}
4


[I 2022-12-17 08:09:29,623] Trial 2 finished with value: 313.79898527853817 and parameters: {'learning_rate': 0.09014138765916972, 'n_estimators': 227, 'colsample_bytree': 0.6695148955243504, 'subsample': 0.6746047873063304, 'reg_alpha': 3.629778394351197, 'reg_lambda': 4.485551299762886}. Best is trial 1 with value: 316.25316112207895.
[I 2022-12-17 08:09:29,713] Trial 3 finished with value: 315.1055167677173 and parameters: {'learning_rate': 0.08882155600224662, 'n_estimators': 256, 'colsample_bytree': 0.8210158230771438, 'subsample': 0.5420699824975244, 'reg_alpha': 0.8081435704730688, 'reg_lambda': 4.492770942635397}. Best is trial 1 with value: 316.25316112207895.
[I 2022-12-17 08:09:29,756] Trial 4 finished with value: 308.4885041015359 and parameters: {'learning_rate': 0.06103647690629941, 'n_estimators': 101, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}. Best is trial 1 with value: 

Best trial : score 343.15878483604905, 
params {'learning_rate': 0.010273802014981157, 'n_estimators': 173, 'colsample_bytree': 0.6326011838408627, 'subsample': 0.6219948216895418, 'reg_alpha': 4.865052773762228, 'reg_lambda': 1.965488623333802}
5


[I 2022-12-17 08:09:30,377] Trial 1 finished with value: 278.2006992672738 and parameters: {'learning_rate': 0.04339023068341412, 'n_estimators': 294, 'colsample_bytree': 0.9818099885446264, 'subsample': 0.92650472773368, 'reg_alpha': 1.4722444603479286, 'reg_lambda': 1.9254886430096263}. Best is trial 0 with value: 281.92986542637993.
[I 2022-12-17 08:09:30,452] Trial 2 finished with value: 285.59237438085853 and parameters: {'learning_rate': 0.08526253048016884, 'n_estimators': 163, 'colsample_bytree': 0.5847463733430462, 'subsample': 0.7784006312291751, 'reg_alpha': 4.680773870803905, 'reg_lambda': 3.480148983374865}. Best is trial 2 with value: 285.59237438085853.
[I 2022-12-17 08:09:30,501] Trial 3 finished with value: 275.08502989516165 and parameters: {'learning_rate': 0.05743605583884714, 'n_estimators': 119, 'colsample_bytree': 0.8075036133495849, 'subsample': 0.9950269250521316, 'reg_alpha': 0.7004200761826201, 'reg_lambda': 2.591648261818684}. Best is trial 2 with value: 285

Best trial : score 285.59237438085853, 
params {'learning_rate': 0.08526253048016884, 'n_estimators': 163, 'colsample_bytree': 0.5847463733430462, 'subsample': 0.7784006312291751, 'reg_alpha': 4.680773870803905, 'reg_lambda': 3.480148983374865}


In [58]:
lgbm = LGBMRegressor()

param_grid = [
    {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], 
     'max_depth': [-1, 15, 30, 45, 60]}
]

grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error', 
                           return_train_score=True, n_jobs=-1)
grid_search.fit(Gangdong_x, Gangdong_y)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_depth': -1, 'n_estimators': 100}
LGBMRegressor()


In [59]:
def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 0, 200),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'subsample' :trial.suggest_float('subsample', 0.5, 1.0),
        'reg_alpha' : trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda' : trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_jobs' : -1,
    }
    model = LGBMRegressor(**param)
    model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0)

    y_valid_pred = model.predict(valid_x)
    return MAE(valid_y, y_valid_pred)

sampler = TPESampler(seed=42)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
n_iter=0
for train_index, test_index in tqdm(kfold.split(Gangdong_x, Gangdong_y)):
    n_iter += 1
    print(n_iter)
    train_x= Gangdong_x.iloc[train_index]
    valid_x= Gangdong_x.iloc[test_index]
    train_y= Gangdong_y.iloc[train_index]
    valid_y= Gangdong_y.iloc[test_index]

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=10)

    print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))
#for Gangdong
#Best trial : score 281.24732501004917, 
#params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}

0it [00:00, ?it/s][I 2022-12-17 08:09:45,132] A new study created in memory with name: no-name-08458083-7170-43cc-ab77-d354193b058a
[I 2022-12-17 08:09:45,196] Trial 0 finished with value: 183.1347089969999 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 191, 'colsample_bytree': 0.8659969709057025, 'subsample': 0.7993292420985183, 'reg_alpha': 0.7800932022121826, 'reg_lambda': 0.7799726016810132}. Best is trial 0 with value: 183.1347089969999.
[I 2022-12-17 08:09:45,245] Trial 1 finished with value: 252.80232891438487 and parameters: {'learning_rate': 0.006750277604651747, 'n_estimators': 174, 'colsample_bytree': 0.8005575058716043, 'subsample': 0.8540362888980227, 'reg_alpha': 0.10292247147901223, 'reg_lambda': 4.8495492608099715}. Best is trial 1 with value: 252.80232891438487.
[I 2022-12-17 08:09:45,270] Trial 2 finished with value: 207.53006691298967 and parameters: {'learning_rate': 0.08341182143924175, 'n_estimators': 42, 'colsample_bytree': 0.5909124836035

1


[I 2022-12-17 08:09:45,353] Trial 4 finished with value: 185.62084240438466 and parameters: {'learning_rate': 0.04615092843748656, 'n_estimators': 157, 'colsample_bytree': 0.5998368910791798, 'subsample': 0.7571172192068059, 'reg_alpha': 2.9620728443102124, 'reg_lambda': 0.23225206359998862}. Best is trial 1 with value: 252.80232891438487.
[I 2022-12-17 08:09:45,374] Trial 5 finished with value: 227.68422435835873 and parameters: {'learning_rate': 0.0611469403382424, 'n_estimators': 34, 'colsample_bytree': 0.5325257964926398, 'subsample': 0.9744427686266666, 'reg_alpha': 4.828160165372797, 'reg_lambda': 4.041986740582305}. Best is trial 1 with value: 252.80232891438487.
[I 2022-12-17 08:09:45,392] Trial 6 finished with value: 302.240940572114 and parameters: {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}. Best is trial 6 with value: 302.2

Best trial : score 302.240940572114, 
params {'learning_rate': 0.0311567631481637, 'n_estimators': 19, 'colsample_bytree': 0.8421165132560784, 'subsample': 0.7200762468698007, 'reg_alpha': 0.6101911742238941, 'reg_lambda': 2.475884550556351}
2


[I 2022-12-17 08:09:45,740] Trial 4 finished with value: 185.38984251780641 and parameters: {'learning_rate': 0.03178724984985056, 'n_estimators': 65, 'colsample_bytree': 0.864803089169032, 'subsample': 0.8187787356776066, 'reg_alpha': 4.436063712881633, 'reg_lambda': 2.3610746258097466}. Best is trial 2 with value: 323.5362098407041.
[I 2022-12-17 08:09:45,788] Trial 5 finished with value: 188.6318430683346 and parameters: {'learning_rate': 0.012839830347891868, 'n_estimators': 143, 'colsample_bytree': 0.8803925243084487, 'subsample': 0.7806385987847482, 'reg_alpha': 3.854835899772805, 'reg_lambda': 2.4689779818219537}. Best is trial 2 with value: 323.5362098407041.
[I 2022-12-17 08:09:45,823] Trial 6 finished with value: 192.7448634071257 and parameters: {'learning_rate': 0.052750550108817414, 'n_estimators': 85, 'colsample_bytree': 0.5127095633720475, 'subsample': 0.5539457134966522, 'reg_alpha': 0.15714592843367126, 'reg_lambda': 3.182052056318902}. Best is trial 2 with value: 323.

...

Best trial : score 380.065830440746, 
params {'learning_rate': 0.029199208943369297, 'n_estimators': 7, 'colsample_bytree': 0.8047821669899484, 'subsample': 0.7513395116144308, 'reg_alpha': 0.25739375624994676, 'reg_lambda': 1.3932323211830573}
4


[I 2022-12-17 08:09:46,682] Trial 5 finished with value: 202.65200537976378 and parameters: {'learning_rate': 0.05532464514729203, 'n_estimators': 139, 'colsample_bytree': 0.8259806297513003, 'subsample': 0.61213465473028, 'reg_alpha': 3.560896106737679, 'reg_lambda': 1.1862454374840004}. Best is trial 4 with value: 453.80620768183275.
[I 2022-12-17 08:09:46,735] Trial 6 finished with value: 205.31067790659833 and parameters: {'learning_rate': 0.03321457011776751, 'n_estimators': 150, 'colsample_bytree': 0.8248164495236073, 'subsample': 0.924611705247089, 'reg_alpha': 3.2880644615017167, 'reg_lambda': 2.841543016677358}. Best is trial 4 with value: 453.80620768183275.
[I 2022-12-17 08:09:46,771] Trial 7 finished with value: 325.48160289387886 and parameters: {'learning_rate': 0.010273802014981157, 'n_estimators': 73, 'colsample_bytree': 0.6326011838408627, 'subsample': 0.6219948216895418, 'reg_alpha': 4.865052773762228, 'reg_lambda': 1.965488623333802}. Best is trial 4 with value: 453.

Best trial : score 453.80620768183275, 
params {'learning_rate': 0.06103647690629941, 'n_estimators': 1, 'colsample_bytree': 0.5507357714330161, 'subsample': 0.8317508845540279, 'reg_alpha': 0.025307919231093434, 'reg_lambda': 0.8040402570874933}
5


[I 2022-12-17 08:09:47,103] Trial 3 finished with value: 236.02554462547045 and parameters: {'learning_rate': 0.05743605583884714, 'n_estimators': 19, 'colsample_bytree': 0.8075036133495849, 'subsample': 0.9950269250521316, 'reg_alpha': 0.7004200761826201, 'reg_lambda': 2.591648261818684}. Best is trial 3 with value: 236.02554462547045.
[I 2022-12-17 08:09:47,157] Trial 4 finished with value: 197.87810267833487 and parameters: {'learning_rate': 0.0878599341208676, 'n_estimators': 148, 'colsample_bytree': 0.848507870497634, 'subsample': 0.8512420419935547, 'reg_alpha': 1.7974557560987758, 'reg_lambda': 1.4679592213224668}. Best is trial 3 with value: 236.02554462547045.
[I 2022-12-17 08:09:47,217] Trial 5 finished with value: 198.24242360911606 and parameters: {'learning_rate': 0.08112675439237285, 'n_estimators': 162, 'colsample_bytree': 0.9335361592900518, 'subsample': 0.9566202762782356, 'reg_alpha': 2.556711994304689, 'reg_lambda': 2.507581473435998}. Best is trial 3 with value: 236

Best trial : score 282.9841658361106, 
params {'learning_rate': 0.03818271231135446, 'n_estimators': 18, 'colsample_bytree': 0.789140070498087, 'subsample': 0.517971136898371, 'reg_alpha': 2.3279900906623006, 'reg_lambda': 2.713223173537883}
